# Validação do Sistema de Pesos

Este notebook valida o sistema de pesos proposto para roteamento:

1. **ISP Usage Weight** (1.0-1.2): Balanceamento interno
2. **Migration Weight** (0.0-0.2): Evitar conflito com migrações
3. **Link Criticality Weight** (0.0-0.4): Proteger recursos críticos

**Range Total:** 1.0 a 1.8


In [1]:
import pickle
from pathlib import Path
from collections import defaultdict

import networkx as nx
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Style
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)


## 1. Carregar Cenário


In [2]:
# Load scenario
from simulador.core.topology import Topology
from simulador.entities.isp import ISP


scenario_path = Path("output/cenario1_copy.pkl")

with open(scenario_path, 'rb') as f:
    cenario = pickle.load(f)

topology: Topology = cenario.topology
lista_de_isps: list[ISP] = topology.lista_de_isps
disaster_node = 9

alfa = 0.2
beta = 0.2
gamma = 0.4


## 2. Calcular ISP Usage Weights

Para cada ISP, calcula a frequência com que cada link aparece em shortest paths internos.


In [3]:
from simulador.entities.isp import ISP


def calculate_isp_usage_weights(isp_list:list[ISP], alfa:float = 0.2):
    """Calculate ISP usage weights based on shortest path frequency."""
    
    link_ocurrances_in_all_isps = defaultdict(int)
    for isp in isp_list:
        
        for edge in isp.edges:
            link_ocurrances_in_all_isps[edge] += 1
            reverse_edge = (edge[1], edge[0])
            link_ocurrances_in_all_isps[reverse_edge] += 1
    
    weights_per_isp = { isp.isp_id: {} for isp in isp_list }
    for isp in isp_list:
        for edge in isp.edges:
            normalized = alfa*(link_ocurrances_in_all_isps[edge] -1)/ (len(isp_list) -1)
            weights_per_isp[isp.isp_id][edge] = normalized
            reverse_edge = (edge[1], edge[0])
            weights_per_isp[isp.isp_id][reverse_edge] = normalized
    
    return weights_per_isp

isp_usage_weights = calculate_isp_usage_weights(lista_de_isps, alfa)



## 3. Calcular Migration Weights

Baseado em quais links são usados pelos paths de migração de datacenters.


In [4]:
def calculate_migration_weights(lista_de_isps: list[ISP], beta: float = 0.2):
    """Calculate migration weights based on datacenter migration paths."""
    
    link_count = defaultdict(int)
    total_migration_paths = 0
    
    for isp in lista_de_isps:
        if not hasattr(isp, 'datacenter') or isp.datacenter is None:
            continue
        
        datacenter = isp.datacenter
        src = datacenter.source
        dst = datacenter.destination
        
        # Get paths from ISP's internal paths
        if hasattr(isp, 'caminhos_internos_isp') and isp.caminhos_internos_isp:
            if src in isp.caminhos_internos_isp and dst in isp.caminhos_internos_isp[src]:
                paths = isp.caminhos_internos_isp[src][dst]
                
                for path_info in paths:
                    caminho = path_info["caminho"]
                    total_migration_paths += 1
                    
                    for i in range(len(caminho) - 1):
                        link = (caminho[i], caminho[i + 1])
                        link_count[link] += 1
                        
                        reverse = (caminho[i + 1], caminho[i])
                        link_count[reverse] += 1
    
    if not link_count:
        return {}
    
    max_count = max(link_count.values())
    migration_weights = {}
    
    for link, count in link_count.items():
        normalized = count / max_count if max_count > 0 else 0
        weight = normalized * beta
        migration_weights[link] = weight
    
    return migration_weights


def calculate_migration_weights_for_visualization(
    lista_de_isps: list[ISP], 
    topology_graph, 
    disaster_node,
    num_paths: int = 5, 
    beta: float = 0.2
):
    """
    Calculate migration weights using K shortest paths within each ISP's subnet.
    
    Args:
        lista_de_isps: List of ISP objects
        topology_graph: Full network topology (with disaster node)
        disaster_node: Node affected by disaster
        num_paths: Number of shortest paths to consider per ISP (K)
        beta: Weight multiplier
    
    Returns:
        Dictionary mapping links to migration weights
    """
    from itertools import islice
    
    link_count = defaultdict(int)
    total_migration_paths = 0
    
    for isp in lista_de_isps:
        if not hasattr(isp, 'datacenter') or isp.datacenter is None:
            continue
        
        datacenter = isp.datacenter
        src = datacenter.source
        dst = datacenter.destination
        
        # Build ISP subgraph (subnet) including disaster node
        isp_nodes = set(isp.nodes)
        isp_subgraph = topology_graph.subgraph(isp_nodes).copy()
        
        # Add ISP edges that might not be in the subgraph
        for edge in isp.edges:
            if (edge[0] in isp_nodes and edge[1] in isp_nodes 
                and not isp_subgraph.has_edge(edge[0], edge[1])
                and topology_graph.has_edge(edge[0], edge[1])):
                edge_data = topology_graph[edge[0]][edge[1]].copy()
                isp_subgraph.add_edge(edge[0], edge[1], **edge_data)
        
        # Calculate K shortest paths within ISP subnet using physical distance
        try:
            paths = list(islice(
                nx.shortest_simple_paths(isp_subgraph, src, dst, weight='weight'),
                num_paths
            ))
            
            for path in paths:
                total_migration_paths += 1
                
                for i in range(len(path) - 1):
                    link = (path[i], path[i + 1])
                    link_count[link] += 1
                    
                    reverse = (path[i + 1], path[i])
                    link_count[reverse] += 1
        except nx.NetworkXNoPath:
            continue
    
    if not link_count:
        return {}
    
    max_count = max(link_count.values())
    migration_weights = {}
    
    for link, count in link_count.items():
        normalized = count / max_count if max_count > 0 else 0
        weight = normalized * beta
        migration_weights[link] = weight
    
    return migration_weights


migration_weights = calculate_migration_weights(lista_de_isps, beta)
migration_weights2 = calculate_migration_weights_for_visualization(lista_de_isps, topology.topology, 9, 6, beta)


In [5]:
migration_weights2 = calculate_migration_weights_for_visualization(lista_de_isps, topology.topology, 9, 7, beta)

migration_weights2[(19, 11)]

0.016666666666666666

## 4. Calcular Link Criticality Weights

Baseado em quantas ISPs dependem de cada link (bridges).


In [6]:
def calculate_link_criticality(topology, disaster_node, gamma: float = 0.4):
    """Calculate link criticality based on bridges across all ISPs."""
    
    link_criticality = {}
    
    for isp in topology.lista_de_isps:
        # Create ISP subgraph
        isp_graph = topology.topology.subgraph(isp.nodes).copy()
        for edge in isp.edges:
            if (edge[0] in isp.nodes and edge[1] in isp.nodes 
                and not isp_graph.has_edge(edge[0], edge[1])
                and topology.topology.has_edge(edge[0], edge[1])):
                edge_data = topology.topology[edge[0]][edge[1]].copy()
                isp_graph.add_edge(edge[0], edge[1], **edge_data)
        
        # Remove disaster node
        if disaster_node in isp_graph.nodes():
            isp_graph.remove_node(disaster_node)
        
        # Find bridges
        bridges = list(nx.bridges(isp_graph))
        
        for link in bridges:
            # Forward direction
            if link not in link_criticality:
                link_criticality[link] = {
                    'bridge_count': 0,
                    'isp_list': [],
                    'weight_penalty': 0.0,
                }
            link_criticality[link]['bridge_count'] += 1
            link_criticality[link]['isp_list'].append(isp.isp_id)
            
            # Reverse direction
            reverse = (link[1], link[0])
            if reverse not in link_criticality:
                link_criticality[reverse] = {
                    'bridge_count': 0,
                    'isp_list': [],
                    'weight_penalty': 0.0,
                }
            link_criticality[reverse]['bridge_count'] += 1
            link_criticality[reverse]['isp_list'].append(isp.isp_id)
    
    
    link_criticality_return = {}
    for link, data in link_criticality.items():
        normalized = (data['bridge_count'] )
        link_criticality_return[link] = normalized/len(lista_de_isps)*gamma
    
    return link_criticality_return


link_criticality_weights = calculate_link_criticality(topology, disaster_node, gamma)

## 5. Exemplo: Calcular Peso Total de um Link

Vamos calcular o peso total para um link específico em diferentes ISPs para validar a lógica condicional.


In [7]:
def calculate_weights_by_isps(
    lista_de_isps: list[ISP],
    isp_usage_weights: dict,
    migration_weights: dict,
    link_criticality_weights: dict
    ) -> dict[int, dict[tuple[int, int], dict[str, float]]]:
    link_weights_by_isp: dict[int, dict[tuple[int, int], dict[str, float]]] = {}
    for isp in lista_de_isps:
        isp_id = isp.isp_id
        link_weights_by_isp[isp.isp_id] = {}
        for link in isp.edges:
            reverse_link = (link[1], link[0])
            isp_weight = isp_usage_weights.get(isp_id, {}).get(link, 0.0)
            if isp_weight == 1.0:  # Try reverse
                isp_weight = isp_usage_weights.get(isp_id, {}).get(reverse_link, 0.0)
            
            migration_weight = migration_weights.get(link, 0.0)
            if migration_weight == 0.0:  # Try reverse
                migration_weight = migration_weights.get(reverse_link, 0.0)
            
            criticality_weight = link_criticality_weights.get(link, 0.0)
            if criticality_weight == 0.0:  # Try reverse
                criticality_weight = link_criticality_weights.get(reverse_link, 0.0)
            
            link_components = {
                    'isp_usage': isp_weight,
                    'migration': migration_weight,
                    'criticality': criticality_weight,
                    'total': 1 + isp_weight + migration_weight + criticality_weight
                }
            total_multiplier = 1 + isp_weight + migration_weight + criticality_weight
            link_weights_by_isp[isp.isp_id][link] = link_components
            link_weights_by_isp[isp.isp_id][reverse_link] = link_components

    return link_weights_by_isp

weights_by_link_by_isp = calculate_weights_by_isps(lista_de_isps, isp_usage_weights, migration_weights, link_criticality_weights)


In [8]:
# ============================================================================

def create_weighted_graph(
    graph: nx.Graph,
    isp_id: int,
    weights_by_link_by_isp: dict[int, dict[tuple[int, int], dict[str, float]]]
) -> nx.Graph:
    """Create a copy of the graph with modified edge weights.
    
    Each edge weight is multiplied by the total weight from 3 components:
    - ISP Usage Weight (0.0-0.alfa)
    - Migration Weight (0.0-0.beta)  
    - Link Criticality Weight (0.0-0.gamma, conditional on ISP)
    
    Args:
        graph: Original NetworkX graph
        isp_id: ID of the ISP requesting paths
        isp_usage_weights: dict[isp_id]['weights'][link] = weight
        migration_weights: dict[link] = weight
        link_criticality_weights: dict[link]['penalty_per_isp'][isp_id] = weight
        
    Returns:
        Graph with modified edge weights
    """
    weighted_graph = graph.copy()
    
    for u, v, data in weighted_graph.edges(data=True):
        link = (u, v)
        
        total_multiplier = weights_by_link_by_isp.get(isp_id, {}).get(link, {}).get('total', 1.0)
        # Apply to edge weight (distance)
        original_weight = data.get('weight', 1.0)
        data['weight'] = original_weight * total_multiplier
    
    return weighted_graph



In [9]:
# ============================================================================
# MIGRATION ANALYSIS VISUALIZATION
# ============================================================================

def visualize_migration_analysis(
    topology_graph,
    lista_de_isps,
    disaster_node,
    max_paths_per_isp=10,
    active_isps=None,
    figsize=(20, 10)
):
    """
    Visualize datacenter migrations across all ISPs.
    
    Left plot: Full topology with all migration paths (with directional arrows)
    Right plot: Statistics about migrations
    
    Note: Migration paths are calculated dynamically using NetworkX shortest_simple_paths
          (same method as Path Explorer). Uses unweighted routing within ISP subnets.
    
    Args:
        topology_graph: NetworkX graph
        lista_de_isps: List of ISP objects
        disaster_node: Node affected by disaster
        max_paths_per_isp: Maximum number of paths to calculate per ISP (default: 10)
        active_isps: Set of active ISP IDs to display (if None, show all)
        figsize: Figure size tuple
    """
    # Collect all migration paths and link usage
    migration_paths = []
    link_usage_count = defaultdict(int)
    directed_link_usage = defaultdict(int)  # Track directed edges (u, v)
    datacenters_info = []
    
    # If no active_isps specified, show all
    if active_isps is None:
        active_isps = set(isp.isp_id for isp in lista_de_isps)
    
    for isp in lista_de_isps:
        # Skip if ISP is not active
        if isp.isp_id not in active_isps:
            continue
            
        if hasattr(isp, 'datacenter') and isp.datacenter is not None:
            datacenter = isp.datacenter
            src = datacenter.source
            dst = datacenter.destination
            
            # Build ISP subgraph (same as Path Explorer)
            isp_subgraph = topology_graph.subgraph(isp.nodes).copy()
            for edge in isp.edges:
                if (edge[0] in isp.nodes and edge[1] in isp.nodes 
                    and not isp_subgraph.has_edge(edge[0], edge[1])
                    and topology_graph.has_edge(edge[0], edge[1])):
                    edge_data = topology_graph[edge[0]][edge[1]].copy()
                    isp_subgraph.add_edge(edge[0], edge[1], **edge_data)
            
            # Calculate k-shortest paths dynamically (same as Path Explorer)
            try:
                from itertools import islice
                paths = list(islice(
                    nx.shortest_simple_paths(isp_subgraph, src, dst, weight='weight'),
                    max_paths_per_isp
                ))
                
                for path in paths:
                    migration_paths.append({
                        'isp_id': isp.isp_id,
                        'source': src,
                        'destination': dst,
                        'path': path
                    })
                    
                    # Count link usage (both undirected and directed)
                    for i in range(len(path) - 1):
                        link = tuple(sorted([path[i], path[i + 1]]))
                        link_usage_count[link] += 1
                        
                        # Track directed edge for arrow drawing
                        directed_edge = (path[i], path[i + 1])
                        directed_link_usage[directed_edge] += 1
                        
            except nx.NetworkXNoPath:
                print(f"Warning: No path found for ISP {isp.isp_id} from {src} to {dst}")
            
            datacenters_info.append({
                'isp_id': isp.isp_id,
                'source': src,
                'destination': dst
            })
    
    # Create figure with two subplots
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=figsize)
    
    # ========== LEFT PLOT: Topology with migration paths ==========
    ax1.set_title('Network Topology - Datacenter Migration Paths', 
                  fontsize=16, fontweight='bold')
    
    # Layout
    pos = nx.spring_layout(topology_graph, seed=7)
    
    # LAYER 1: Background nodes
    nx.draw_networkx_nodes(
        topology_graph, pos,
        node_color='lightgray',
        node_size=500,
        edgecolors='black',
        linewidths=1,
        alpha=0.5,
        ax=ax1
    )
    
    # LAYER 2: Background edges (light gray)
    nx.draw_networkx_edges(
        topology_graph, pos,
        edge_color='lightgray',
        width=1,
        alpha=0.3,
        ax=ax1
    )
    
    # LAYER 3: Migration path edges with directional arrows (colored by usage frequency)
    if directed_link_usage:
        max_usage = max(directed_link_usage.values())
        
        # Draw migration links with thickness based on usage and arrows for direction
        for directed_edge, count in directed_link_usage.items():
            u, v = directed_edge
            if topology_graph.has_edge(u, v) or topology_graph.has_edge(v, u):
                width = 2 + (count / max_usage) * 8  # Range: 2-10
                alpha = 0.5 + (count / max_usage) * 0.4  # Range: 0.5-0.9
                
                nx.draw_networkx_edges(
                    topology_graph, pos,
                    edgelist=[(u, v)],
                    edge_color='steelblue',
                    width=width,
                    alpha=alpha,
                    arrows=True,
                    arrowsize=15 + (count / max_usage) * 10,  # Range: 15-25
                    arrowstyle='->',
                    connectionstyle='arc3,rad=0.1',
                    ax=ax1
                )
    
    # LAYER 4: Source nodes (datacenter sources)
    source_nodes = [dc['source'] for dc in datacenters_info]
    if source_nodes:
        nx.draw_networkx_nodes(
            topology_graph, pos,
            nodelist=source_nodes,
            node_color='orange',
            node_size=800,
            edgecolors='darkorange',
            linewidths=3,
            ax=ax1,
            label='Datacenter Source'
        )
    
    # LAYER 5: Destination nodes (datacenter destinations)
    dest_nodes = [dc['destination'] for dc in datacenters_info]
    if dest_nodes:
        nx.draw_networkx_nodes(
            topology_graph, pos,
            nodelist=dest_nodes,
            node_color='limegreen',
            node_size=800,
            edgecolors='darkgreen',
            linewidths=3,
            ax=ax1,
            label='Datacenter Destination'
        )
    
    # LAYER 6: Disaster node (red X)
    if disaster_node in topology_graph.nodes():
        nx.draw_networkx_nodes(
            topology_graph, pos,
            nodelist=[disaster_node],
            node_color='red',
            node_size=1000,
            node_shape='X',
            edgecolors='darkred',
            linewidths=3,
            ax=ax1,
            label='Disaster Node'
        )
    
    # Node labels
    nx.draw_networkx_labels(
        topology_graph, pos,
        font_size=9,
        font_weight='bold',
        ax=ax1
    )
    
    ax1.axis('off')
    ax1.legend(loc='upper right', fontsize=11, framealpha=0.9)
    
    # ========== RIGHT PLOT: Migration statistics ==========
    ax2.set_title('Migration Statistics', fontsize=16, fontweight='bold')
    ax2.axis('off')
    
    # Summary
    y_pos = 0.95
    ax2.text(0.5, y_pos, 'Datacenter Migrations Summary',
            ha='center', va='top', fontsize=14, fontweight='bold',
            transform=ax2.transAxes,
            bbox=dict(boxstyle='round', facecolor='lightblue', alpha=0.8))
    
    y_pos -= 0.1
    ax2.text(0.5, y_pos, f'Total ISPs with datacenters: {len(datacenters_info)}',
            ha='center', va='top', fontsize=12,
            transform=ax2.transAxes)
    
    y_pos -= 0.05
    ax2.text(0.5, y_pos, f'Total migration paths: {len(migration_paths)}',
            ha='center', va='top', fontsize=12,
            transform=ax2.transAxes)
    
    y_pos -= 0.05
    ax2.text(0.5, y_pos, f'Unique links used: {len(link_usage_count)}',
            ha='center', va='top', fontsize=12,
            transform=ax2.transAxes)
    
    # Most used links
    if link_usage_count:
        y_pos -= 0.1
        ax2.text(0.1, y_pos, 'Most Used Links:',
                fontsize=12, fontweight='bold',
                transform=ax2.transAxes)
        
        # Top 10 links
        sorted_links = sorted(link_usage_count.items(), key=lambda x: x[1], reverse=True)[:10]
        y_pos -= 0.06
        for link, count in sorted_links:
            ax2.text(0.15, y_pos, f'{link[0]} ↔ {link[1]}: {count} paths',
                    fontsize=10, family='monospace',
                    transform=ax2.transAxes)
            y_pos -= 0.045
    
    # ISP-specific migrations
    if datacenters_info:
        y_pos -= 0.05
        ax2.text(0.1, y_pos, 'Migrations by ISP:',
                fontsize=12, fontweight='bold',
                transform=ax2.transAxes)
        
        y_pos -= 0.06
        for dc in datacenters_info:
            isp_paths = [p for p in migration_paths if p['isp_id'] == dc['isp_id']]
            ax2.text(0.15, y_pos, 
                    f"ISP {dc['isp_id']}: {dc['source']} → {dc['destination']} ({len(isp_paths)} paths)",
                    fontsize=10, family='monospace',
                    transform=ax2.transAxes)
            y_pos -= 0.045
    
    plt.tight_layout()
    plt.show()
    
    return migration_paths, link_usage_count, directed_link_usage


In [10]:
# ============================================================================
# ISP USAGE ANALYSIS VISUALIZATION
# ============================================================================

def visualize_isp_usage_analysis(
    topology_graph,
    lista_de_isps,
    selected_link=None,
    figsize=(20, 10)
):
    """
    Visualize ISP usage across the network.
    
    Left plot: Full topology with edges colored by number of ISPs using them
    Right plot: Details about selected link showing which ISPs use it
    
    Args:
        topology_graph: NetworkX graph
        lista_de_isps: List of ISP objects
        selected_link: Tuple (u, v) representing selected link
        figsize: Figure size tuple
    """
    # Calculate ISP usage per link
    link_isp_count = defaultdict(set)
    
    for isp in lista_de_isps:
        for edge in isp.edges:
            # Normalize edge direction
            normalized_edge = tuple(sorted(edge))
            link_isp_count[normalized_edge].add(isp.isp_id)
    
    # Create figure with two subplots
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=figsize)
    
    # ========== LEFT PLOT: Topology with ISP usage coloring ==========
    ax1.set_title('Network Topology - Link Sharing Across ISPs', 
                  fontsize=16, fontweight='bold')
    
    # Layout
    pos = nx.spring_layout(topology_graph, seed=7)
    
    # Draw all nodes
    nx.draw_networkx_nodes(
        topology_graph, pos,
        node_color='lightgray',
        node_size=600,
        edgecolors='black',
        linewidths=1.5,
        ax=ax1
    )
    
    # Draw edges colored by ISP count
    edge_colors = []
    edge_widths = []
    edges_to_draw = []
    
    for u, v in topology_graph.edges():
        normalized_edge = tuple(sorted([u, v]))
        isp_count = len(link_isp_count.get(normalized_edge, set()))
        
        edges_to_draw.append((u, v))
        edge_colors.append(isp_count)
        edge_widths.append(2 + isp_count * 2)  # Thicker = more ISPs
    
    # Color map
    max_isps = max(edge_colors) if edge_colors else 1
    cmap = plt.colormaps['YlOrRd']
    
    edges_collection = nx.draw_networkx_edges(
        topology_graph, pos,
        edgelist=edges_to_draw,
        edge_color=edge_colors,
        edge_cmap=cmap,
        edge_vmin=0,
        edge_vmax=max_isps,
        width=edge_widths,
        alpha=0.8,
        ax=ax1
    )
    
    # Highlight selected link if provided
    if selected_link:
        normalized_selected = tuple(sorted(selected_link))
        # Draw selected link in bright green with thick border
        nx.draw_networkx_edges(
            topology_graph, pos,
            edgelist=[selected_link],
            edge_color='lime',
            width=8,
            alpha=1.0,
            ax=ax1,
            style='solid'
        )
    
    # Node labels
    nx.draw_networkx_labels(
        topology_graph, pos,
        font_size=10,
        font_weight='bold',
        ax=ax1
    )
    
    # Colorbar
    sm = plt.cm.ScalarMappable(cmap=cmap, 
                               norm=plt.Normalize(vmin=0, vmax=max_isps))
    sm.set_array([])
    cbar = plt.colorbar(sm, ax=ax1, fraction=0.046, pad=0.04)
    cbar.set_label('Number of ISPs Sharing Link', rotation=270, labelpad=20, fontweight='bold')
    
    ax1.axis('off')
    ax1.legend([plt.Line2D([0], [0], color='lime', linewidth=4)],
               ['Selected Link'],
               loc='upper right', fontsize=10)
    
    # ========== RIGHT PLOT: Link details ==========
    ax2.set_title('Link Details', fontsize=16, fontweight='bold')
    ax2.axis('off')
    
    if selected_link:
        normalized_selected = tuple(sorted(selected_link))
        isps_using_link = sorted(link_isp_count.get(normalized_selected, set()))
        
        if isps_using_link:
            # Link header
            ax2.text(0.5, 0.95, f'Link: {selected_link[0]} ↔ {selected_link[1]}',
                    ha='center', va='top', fontsize=16, fontweight='bold',
                    transform=ax2.transAxes,
                    bbox=dict(boxstyle='round', facecolor='lightblue', alpha=0.8))
            
            # Total ISPs count
            ax2.text(0.5, 0.85, f'Total ISPs sharing this link: {len(isps_using_link)}',
                    ha='center', va='top', fontsize=13, fontweight='bold',
                    transform=ax2.transAxes)
            
            # List ISPs with details
            y_position = 0.75
            ax2.text(0.1, y_position, 'ISPs using this link:',
                    fontsize=12, fontweight='bold',
                    transform=ax2.transAxes)
            
            y_position -= 0.08
            for isp_id in isps_using_link:
                isp = lista_de_isps[isp_id]
                # ISP header
                ax2.text(0.1, y_position, f'• ISP {isp_id}',
                        fontsize=11, fontweight='bold',
                        transform=ax2.transAxes)
                y_position -= 0.05
                
                # ISP details
                ax2.text(0.15, y_position, 
                        f'{len(isp.nodes)} nodes, {len(isp.edges)} edges',
                        fontsize=10, style='italic', color='dimgray',
                        transform=ax2.transAxes, family='monospace')
                y_position -= 0.07
        else:
            ax2.text(0.5, 0.5, 
                    f'Link {selected_link[0]} ↔ {selected_link[1]}\n\n'
                    'Not used by any ISP',
                    ha='center', va='center', fontsize=13,
                    transform=ax2.transAxes)
    else:
        ax2.text(0.5, 0.5, 
                'Select a link from the dropdown\nto see which ISPs use it',
                ha='center', va='center', fontsize=13, style='italic',
                transform=ax2.transAxes)
    
    plt.tight_layout()
    plt.show()
    
    # Return link statistics
    return link_isp_count


In [11]:
# ============================================================================
# LINK CRITICALITY ANALYSIS VISUALIZATION
# ============================================================================

def visualize_link_criticality_analysis(
    topology_graph,
    lista_de_isps,
    disaster_node,
    weights_by_link_by_isp,
    disaster_mode=False,
    figsize=(20, 10)
):
    """
    Visualize link criticality across the network.
    
    Left plot: Bridges colored by gamma weight (yellow→orange→red), non-bridges grey
    Right plot: Default = Bridges ranked by ISP impact, or detailed link analysis
    
    Bridge coloring: Uses actual gamma weight values (not normalized), so color
    intensity reflects the actual penalty and varies with the γ parameter setting.
    
    Bridge Detection: Performed at ISP subnet level, not global network level.
    A link is marked as a bridge if it's a bridge for at least one ISP's allocated
    resources. Can be checked in either normal mode (with disaster node) or 
    disaster mode (without disaster node).
    
    Args:
        topology_graph: NetworkX graph
        lista_de_isps: List of ISP objects
        disaster_node: Node affected by disaster
        weights_by_link_by_isp: Dictionary of weight components by link and ISP
        selected_link: Tuple (u, v) for selected link to analyze (None = show ranking)
        show_bridges_only: If True, only show bridge edges (hide grey non-bridges)
        min_criticality: Minimum gamma weight threshold to display
        disaster_mode: If True, analyze with disaster node removed
        figsize: Figure size tuple
    """
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=figsize)
    
    # Use consistent layout
    pos = nx.spring_layout(topology_graph, seed=7, k=0.3)
    
    # ========== Calculate link criticality scores ==========
    link_criticality = defaultdict(float)
    link_isp_usage = defaultdict(set)
    
    # weights_by_link_by_isp is actually current_weights dict
    if 'link_criticality' in weights_by_link_by_isp:
        for link, crit_weight in weights_by_link_by_isp['link_criticality'].items():
            normalized_link = tuple(sorted(link))
            link_criticality[normalized_link] = crit_weight  # Set directly, don't accumulate
            # For ISP usage, we need to check which ISPs use this link
            for isp in lista_de_isps:
                for edge in isp.edges:
                    if tuple(sorted(edge)) == normalized_link:
                        link_isp_usage[normalized_link].add(isp.isp_id)
    
    # ========== Detect bridges per ISP ==========
    # Bridge detection at ISP subnet level (not global network level)
    bridges = set()  # Links that are bridges for at least one ISP
    bridges_by_isp = defaultdict(set)  # Track which ISPs consider each link a bridge
    
    for isp in lista_de_isps:
        if disaster_mode:
            # Disaster mode: with disaster node removed
            isp_nodes = [n for n in isp.nodes if n != disaster_node]
        else:
            # Normal mode: with disaster node included
            isp_nodes = list(isp.nodes)
        
        if len(isp_nodes) < 2:
            continue
            
        # Build ISP subgraph
        isp_subgraph = topology_graph.subgraph(isp_nodes).copy()
        
        # Add ISP edges to ensure we have the full ISP topology
        for edge in isp.edges:
            if (edge[0] in isp_nodes and edge[1] in isp_nodes 
                and topology_graph.has_edge(edge[0], edge[1])):
                if not isp_subgraph.has_edge(edge[0], edge[1]):
                    edge_data = topology_graph[edge[0]][edge[1]].copy()
                    isp_subgraph.add_edge(edge[0], edge[1], **edge_data)
        
        # Detect bridges in ISP subnet
        if nx.is_connected(isp_subgraph):
            isp_bridges = list(nx.bridges(isp_subgraph))
            for bridge in isp_bridges:
                normalized_bridge = tuple(sorted(bridge))
                bridges.add(normalized_bridge)
                bridges_by_isp[normalized_bridge].add(isp.isp_id)
    
    # ========== LEFT PLOT: Criticality Heatmap ==========
    mode_text = 'Disaster Mode' if disaster_mode else 'Normal Mode'
    ax1.set_title(f'Network Topology - Link Criticality Heatmap [{mode_text}]', 
                  fontsize=16, fontweight='bold')
    
    # Draw base topology (nodes)
    nx.draw_networkx_nodes(
        topology_graph, pos,
        node_color='lightblue',
        node_size=300,
        ax=ax1
    )
    
    # Draw disaster node with X marker if disaster mode is active
    if disaster_mode and disaster_node in topology_graph.nodes():
        nx.draw_networkx_nodes(
            topology_graph, pos,
            nodelist=[disaster_node],
            node_color='red',
            node_size=1000,
            node_shape='X',
            edgecolors='darkred',
            linewidths=3,
            ax=ax1,
            label='Disaster Node'
        )
    
    # Draw node labels
    nx.draw_networkx_labels(
        topology_graph, pos,
        font_size=8,
        ax=ax1
    )
    
    # Separate edges into bridges and non-bridges
    bridge_edges = []
    bridge_colors = []
    bridge_widths = []
    
    non_bridge_edges = []
    
    max_criticality = max(link_criticality.values()) if link_criticality else 1.0
    
    for u, v in topology_graph.edges():
        normalized_edge = tuple(sorted([u, v]))
        crit_score = link_criticality.get(normalized_edge, 0.0)
        is_bridge = normalized_edge in bridges
        
        # Apply filters
        if is_bridge:
            # Bridge edges: color by gamma weight
            bridge_edges.append((u, v))
            # Use actual criticality score (not normalized)
            bridge_colors.append(crit_score)
            
            # Width by number of ISPs affected
            num_isps = len(bridges_by_isp.get(normalized_edge, set()))
            bridge_widths.append(3 + num_isps * 2)
        else:
            # Non-bridge edges: grey
            non_bridge_edges.append((u, v))
    
    # Draw non-bridge edges in grey
    if non_bridge_edges:
        nx.draw_networkx_edges(
            topology_graph, pos,
            edgelist=non_bridge_edges,
            edge_color='lightgray',
            width=1.5,
            alpha=0.5,
            ax=ax1
        )
    
    # Draw bridge edges with criticality colors
    if bridge_edges:
        # Use YlOrRd colormap but scale by actual gamma values
        edge_collection = nx.draw_networkx_edges(
            topology_graph, pos,
            edgelist=bridge_edges,
            edge_color=bridge_colors,
            width=bridge_widths,
            edge_cmap=plt.cm.YlOrRd,  # Yellow -> Orange -> Red
            edge_vmin=0,
            edge_vmax=max_criticality,
            ax=ax1
        )
    
    # Legend
    from matplotlib.lines import Line2D
    legend_elements = [
        Line2D([0], [0], color='darkred', linewidth=4, 
               label=f'Bridges ({len(bridges)} found) - colored by γ weight'),
        Line2D([0], [0], color='lightgray', linewidth=2, 
               label='Non-bridge links'),
    ]
    if max_criticality > 0:
        legend_elements.insert(1, 
            Line2D([0], [0], color='white', linewidth=0,
                   label=f'γ range: 0.0 to {max_criticality:.3f}'))
    if disaster_mode and disaster_node in topology_graph.nodes():
        legend_elements.append(
            Line2D([0], [0], marker='X', color='w', markerfacecolor='red', 
                   markeredgecolor='darkred', markeredgewidth=2, markersize=12,
                   label='Disaster Node', linestyle='None'))
    ax1.legend(handles=legend_elements, loc='upper left', fontsize=9)
    
    ax1.axis('off')
    
    # ========== RIGHT PLOT: Link Detail Analysis ==========
    ax2.set_title('Link Detail Analysis', fontsize=16, fontweight='bold')
    ax2.axis('off')
    
    # Show bridges sorted by ISP count
    y_pos = 0.95
    ax2.text(0.5, y_pos, f'Critical Bridges - {mode_text}',
            ha='center', va='top', fontsize=14, fontweight='bold',
            transform=ax2.transAxes,
            bbox=dict(boxstyle='round', facecolor='lightcoral', alpha=0.8))
    
    y_pos -= 0.10
    
    if bridges:
        # Sort bridges by number of ISPs (descending)
        sorted_bridges = sorted(bridges_by_isp.items(), 
                               key=lambda x: len(x[1]), 
                               reverse=True)
        
        ax2.text(0.1, y_pos, f'Found {len(bridges)} bridge(s) across {len(lista_de_isps)} ISPs',
                fontsize=11, fontweight='bold', transform=ax2.transAxes)
        y_pos -= 0.06
        ax2.text(0.1, y_pos, 'Ranked by number of affected ISPs:',
                fontsize=10, style='italic', color='gray',
                transform=ax2.transAxes)
        y_pos -= 0.08
        
        # Show top 10 bridges with enhanced information
        for i, (bridge, isp_ids) in enumerate(sorted_bridges[:10], 1):
            num_isps = len(isp_ids)
            color = 'darkred' if num_isps >= 3 else ('red' if num_isps == 2 else 'orange')
            
            # Get criticality weight for this bridge
            crit_weight = link_criticality.get(bridge, 0.0)
            
            ax2.text(0.1, y_pos, f'{i}. Link {bridge[0]} ↔ {bridge[1]}',
                    fontsize=10, fontweight='bold', color=color,
                    transform=ax2.transAxes)
            y_pos -= 0.04
            
            # Criticality weight
            ax2.text(0.15, y_pos, f'   Criticality Weight (γ): {crit_weight:.3f}',
                    fontsize=9, color=color,
                    transform=ax2.transAxes)
            y_pos -= 0.04
            
            # Critical ISPs
            ax2.text(0.15, y_pos, f'   Critical for ISPs: {sorted(isp_ids)}',
                    fontsize=9, color=color,
                    transform=ax2.transAxes)
            y_pos -= 0.04
            
            # Summary
            ax2.text(0.15, y_pos, f'   Affects {num_isps} ISP(s)',
                    fontsize=9, color=color, style='italic',
                    transform=ax2.transAxes)
            y_pos -= 0.06
            
            if i == 10 and len(sorted_bridges) > 10:
                ax2.text(0.1, y_pos, f'... and {len(sorted_bridges) - 10} more',
                        fontsize=9, style='italic', color='gray',
                        transform=ax2.transAxes)
                break
    else:
        ax2.text(0.5, 0.5, f'No bridges found in {mode_text}',
                ha='center', va='center', fontsize=12, color='green',
                transform=ax2.transAxes)
        y_pos -= 0.08
        ax2.text(0.5, 0.4, 'All links have redundant paths!',
                ha='center', va='center', fontsize=11, style='italic',
                color='gray', transform=ax2.transAxes)
    
    plt.subplots_adjust(left=0.05, right=0.95, top=0.9, bottom=0.1, wspace=0.3)
    plt.show()
    
    # Return statistics
    return {
        'bridges': bridges,
        'bridges_by_isp': dict(bridges_by_isp),
        'link_criticality': link_criticality,
        'max_criticality': max_criticality
    }


In [12]:
# ============================================================================
# SEPARATED VISUALIZATION FUNCTIONS
# ============================================================================

def plot_isp_topology(
    ax,
    isp,
    topology_graph,
    disaster_node,
    isp_id,
    edge_weights,
    link_frequency,
    remove_disaster_node=True
):
    """
    Plot ISP topology on the left side (colored by weights, sized by frequency).
    
    Args:
        ax: Matplotlib axis to plot on
        isp: ISP object
        topology_graph: Full network topology
        disaster_node: Node affected by disaster
        isp_id: ISP ID
        edge_weights: Dict of link -> total weight
        link_frequency: Dict of link -> frequency in shortest paths
        remove_disaster_node: If True, removes disaster node from graph (default: True)
    """
    # Get ISP nodes
    isp_nodes_set = set(isp.nodes)
    isp_nodes_active = isp_nodes_set - {disaster_node} if remove_disaster_node else isp_nodes_set
    
    # Build ISP graph
    isp_graph = topology_graph.subgraph(isp_nodes_active).copy()
    for edge in isp.edges:
        if (edge[0] in isp_nodes_active and edge[1] in isp_nodes_active 
            and not isp_graph.has_edge(edge[0], edge[1])
            and topology_graph.has_edge(edge[0], edge[1])):
            edge_data = topology_graph[edge[0]][edge[1]].copy()
            isp_graph.add_edge(edge[0], edge[1], **edge_data)
    
    is_connected = nx.is_connected(isp_graph)
    components = list(nx.connected_components(isp_graph)) if not is_connected else []
    
    # Title
    view_mode = 'Disaster View' if remove_disaster_node else 'Normal View'
    title = f'ISP {isp_id} - Topologia [{view_mode}] '
    title += '(PARTICIONADA)' if not is_connected else 'com Pesos'
    ax.set_title(title, fontsize=16, fontweight='bold')
    
    # Layout (consistent with full topology)
    pos = nx.spring_layout(topology_graph, seed=7)
    
    # Background nodes
    all_nodes = list(topology_graph.nodes())
    background_nodes = [n for n in all_nodes if n not in isp_nodes_set and n != disaster_node]
    
    # LAYER 1: Background topology (gray)
    if background_nodes:
        nx.draw_networkx_nodes(
            topology_graph, pos,
            nodelist=background_nodes,
            node_color='lightgray',
            node_size=500,
            edgecolors='gray',
            linewidths=1,
            alpha=0.3,
            ax=ax
        )
    
    all_edges = list(topology_graph.edges())
    nx.draw_networkx_edges(
        topology_graph, pos,
        edgelist=all_edges,
        edge_color='lightgray',
        width=1,
        alpha=0.3,
        ax=ax
    )
    
    # LAYER 2: ISP nodes (highlighted)
    isp_nodes_list = list(isp_nodes_active)
    if isp_nodes_list:
        nx.draw_networkx_nodes(
            topology_graph, pos,
            nodelist=isp_nodes_list,
            node_color='lightblue',
            node_size=800,
            edgecolors='black',
            linewidths=2,
            ax=ax
        )
    
    # Datacenter destination node
    if isp.datacenter.destination:
        nx.draw_networkx_nodes(
            topology_graph, pos,
            nodelist=[isp.datacenter.destination],
            node_color='lightgreen',
            node_size=800,
            edgecolors='black',
            linewidths=2,
            ax=ax
        )
    
    # LAYER 3: Disaster node (red X) - only in disaster view
    if remove_disaster_node and disaster_node in isp_nodes_set:
        nx.draw_networkx_nodes(
            topology_graph, pos,
            nodelist=[disaster_node],
            node_color='red',
            node_size=1000,
            node_shape='X',
            edgecolors='darkred',
            linewidths=3,
            ax=ax,
            label='Nó do Desastre'
        )
    
    # Node labels
    nx.draw_networkx_labels(
        topology_graph, pos,
        font_size=9,
        font_weight='bold',
        ax=ax
    )
    
    # LAYER 4: ISP edges (colored by weight, thickness by frequency)
    edges = list(isp_graph.edges())
    weights_list = [edge_weights.get((u, v), 1.0) for u, v in edges]
    
    # Calculate widths based on frequency
    max_freq = max(link_frequency.values()) if link_frequency else 1
    widths = []
    for u, v in edges:
        link = tuple(sorted([u, v]))
        freq = link_frequency.get(link, 0)
        width = 2 + (freq / max_freq) * 6 if max_freq > 0 else 3
        widths.append(width)
    
    # Color mapping
    vmin = 1.0
    vmax = max(weights_list) if weights_list else 1.8
    
    nx.draw_networkx_edges(
        topology_graph, pos,
        edgelist=edges,
        edge_color=weights_list,
        edge_cmap=plt.cm.RdYlGn_r,
        edge_vmin=vmin,
        edge_vmax=vmax,
        width=widths,
        alpha=0.8,
        ax=ax
    )
    
    # Colorbar
    sm = plt.cm.ScalarMappable(
        cmap=plt.cm.RdYlGn_r, 
        norm=plt.Normalize(vmin=vmin, vmax=vmax)
    )
    sm.set_array([])
    cbar = plt.colorbar(sm, ax=ax, fraction=0.046, pad=0.04)
    cbar.set_label('Peso Total', rotation=270, labelpad=20, fontsize=12)
    
    # Partition warning
    if not is_connected:
        ax.text(0.5, 0.05, f'[!] Rede particionada em {len(components)} componentes', 
                transform=ax.transAxes,
                ha='center', fontsize=11, color='red', fontweight='bold',
                bbox=dict(boxstyle='round', facecolor='yellow', alpha=0.7))
    
    ax.axis('off')
    if remove_disaster_node and disaster_node in isp_nodes_set:
        ax.legend(loc='upper left', fontsize=10)
    
    return is_connected, components


def plot_weight_decomposition(
    ax,
    isp_id,
    edge_weights,
    edge_components,
    is_connected,
    components,
    top_n=10
):
    """
    Plot weight decomposition bar chart on the right side.
    
    Args:
        ax: Matplotlib axis to plot on
        isp_id: ISP ID
        edge_weights: Dict of link -> total weight
        edge_components: Dict of link -> {isp_usage, migration, criticality, total}
        is_connected: Whether ISP is connected
        components: List of connected components (if partitioned)
        top_n: Number of top links to show
    """
    if not edge_weights:
        # Fallback: No weights
        ax.set_title(f'ISP {isp_id} - Sem Dados de Pesos', 
                      fontsize=16, fontweight='bold')
        ax.text(0.5, 0.5, 
                'Pesos não foram calculados para esta ISP',
                ha='center', va='center', fontsize=12,
                transform=ax.transAxes)
        ax.axis('off')
        return
    
    # Title
    title = f'ISP {isp_id} - Top {top_n} Links por Peso (Únicos)'
    if not is_connected:
        title += f' ({len(components)} Componentes)'
    ax.set_title(title, fontsize=16, fontweight='bold')
    
    # Normalize edges to avoid duplicates (always store as (min, max))
    unique_edges = {}
    for (u, v), weight in edge_weights.items():
        normalized_edge = (min(u, v), max(u, v))
        if normalized_edge not in unique_edges:
            unique_edges[normalized_edge] = weight
    
    # Get top N links by weight
    sorted_edges = sorted(unique_edges.items(), key=lambda x: x[1], reverse=True)[:top_n]
    
    if not sorted_edges:
        ax.text(0.5, 0.5, 'Sem links para mostrar', 
                ha='center', va='center', fontsize=12,
                transform=ax.transAxes)
        ax.axis('off')
        return
    
    link_names = [f"{u}↔{v}" for (u, v), _ in sorted_edges]  # Use ↔ to show undirected
    
    # Extract weight components (try both directions since edge_components may have either)
    def get_edge_component(link, component_name):
        """Get component value, trying both edge directions."""
        if link in edge_components:
            return edge_components[link][component_name]
        # Try reverse direction
        reverse_link = (link[1], link[0])
        if reverse_link in edge_components:
            return edge_components[reverse_link][component_name]
        return 0.0
    
    isp_usage_vals = [get_edge_component(link, 'isp_usage') for link, _ in sorted_edges]
    migration_vals = [get_edge_component(link, 'migration') for link, _ in sorted_edges]
    criticality_vals = [get_edge_component(link, 'criticality') for link, _ in sorted_edges]
    
    x = range(len(link_names))
    bar_height = 0.6
    
    # Stacked bar chart - build cumulative positions
    base_vals = [1.0] * len(x)
    left_isp = base_vals
    left_migration = [base + isp for base, isp in zip(base_vals, isp_usage_vals)]
    left_criticality = [base + isp + mig for base, isp, mig in zip(base_vals, isp_usage_vals, migration_vals)]
    
    # Draw stacked bars
    ax.barh(x, base_vals, bar_height, label='Base (1.0)', color='lightgray', edgecolor='black', linewidth=0.5)
    ax.barh(x, isp_usage_vals, bar_height, left=left_isp, 
             label='ISP Usage', color='skyblue', edgecolor='black', linewidth=0.5)
    ax.barh(x, migration_vals, bar_height, left=left_migration,
             label='Migration', color='lightgreen', edgecolor='black', linewidth=0.5)
    ax.barh(x, criticality_vals, bar_height, left=left_criticality,
             label='Criticality', color='salmon', edgecolor='black', linewidth=0.5)
    
    # Formatting
    ax.set_yticks(x)
    ax.set_yticklabels(link_names, fontsize=10)
    ax.set_xlabel('Peso Total', fontsize=12, fontweight='bold')
    ax.set_ylabel('Link', fontsize=12, fontweight='bold')
    vmax = max(w for w in edge_weights.values())
    ax.set_xlim(0.9, vmax * 1.05)
    ax.legend(loc='upper right', fontsize=10, framealpha=0.9)
    ax.grid(axis='x', alpha=0.3, linestyle='--')
    
    # Add total weight values
    for i, (link, total_weight) in enumerate(sorted_edges):
        ax.text(total_weight + 0.02, i, f'{total_weight:.2f}', 
                va='center', fontsize=9, fontweight='bold')
    
    # Add note about unique edges
    ax.text(0.98, 0.02, 'Links únicos (↔ = não-direcionais)', 
            transform=ax.transAxes, fontsize=8,
            ha='right', va='bottom', style='italic',
            bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))


# ============================================================================
# MAIN WRAPPER FUNCTION (combines both plots)
# ============================================================================

def visualize_isp_topology_with_weights(
    isp, 
    topology_graph, 
    disaster_node,
    isp_id,
    weights_by_link_by_isp,
    figsize=(18, 10),
    remove_disaster_node=True
):
    """
    Visualize ISP topology with weights (wrapper for two separate plots).
    
    Args:
        isp: ISP object
        topology_graph: Main topology graph (full network)
        disaster_node: Node to remove (disaster)
        isp_id: ID of the ISP
        weights_by_link_by_isp: Computed weights for all ISPs
        figsize: Figure size tuple
        remove_disaster_node: If True, removes disaster node from visualization (default: True)
    """
    # === PREPARE DATA ===
    isp_nodes_set = set(isp.nodes)
    isp_nodes_active = isp_nodes_set - {disaster_node} if remove_disaster_node else isp_nodes_set
    
    # Build ISP graph
    isp_graph = topology_graph.subgraph(isp_nodes_active).copy()
    for edge in isp.edges:
        if (edge[0] in isp_nodes_active and edge[1] in isp_nodes_active 
            and not isp_graph.has_edge(edge[0], edge[1])
            and topology_graph.has_edge(edge[0], edge[1])):
            edge_data = topology_graph[edge[0]][edge[1]].copy()
            isp_graph.add_edge(edge[0], edge[1], **edge_data)
    
    is_connected = nx.is_connected(isp_graph)
    components = list(nx.connected_components(isp_graph)) if not is_connected else []
    
    if not is_connected:
        pass
    
    # Calculate link frequency (for thickness)
    link_frequency = defaultdict(int)
    if is_connected:
        nodes = sorted(isp_graph.nodes())
        for src in nodes:
            for dst in nodes:
                if src >= dst:
                    continue
                try:
                    path = nx.shortest_path(isp_graph, src, dst, weight='weight')
                    for i in range(len(path) - 1):
                        link = tuple(sorted([path[i], path[i + 1]]))
                        link_frequency[link] += 1
                except nx.NetworkXNoPath:
                    continue
    else:
        for component in nx.connected_components(isp_graph):
            comp_nodes = sorted(component)
            for src in comp_nodes:
                for dst in comp_nodes:
                    if src >= dst:
                        continue
                    try:
                        path = nx.shortest_path(isp_graph, src, dst, weight='weight')
                        for i in range(len(path) - 1):
                            link = tuple(sorted([path[i], path[i + 1]]))
                            link_frequency[link] += 1
                    except nx.NetworkXNoPath:
                        continue
    
    # Calculate weights
    edge_weights = {}
    edge_components = {}
    for u, v in isp_graph.edges():
        for link in [(u, v), (v, u)]:
            if isp_id in weights_by_link_by_isp and weights_by_link_by_isp[isp_id]:
                weights = weights_by_link_by_isp.get(isp_id, {}).get(link, {})
                edge_weights[link] = weights['total']
                edge_components[link] = weights
            else:
                edge_weights[link] = 1.0
                edge_components[link] = {
                    'isp_usage': 1.0,
                    'migration': 0.0,
                    'criticality': 0.0,
                    'total': 1.0
                }
    
    # === CREATE FIGURE WITH TWO PLOTS ===
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=figsize)
    
    # LEFT PLOT: Topology
    is_connected, components = plot_isp_topology(
        ax1,
        isp,
        topology_graph,
        disaster_node,
        isp_id,
        edge_weights,
        link_frequency,
        remove_disaster_node=remove_disaster_node
    )
    
    # RIGHT PLOT: Weight Decomposition
    plot_weight_decomposition(
        ax2,
        isp_id,
        edge_weights,
        edge_components,
        is_connected,
        components,
        top_n=len(isp.edges)

    )
    
    plt.tight_layout()
    plt.show()
    
    # Print statistics
    print(f"\n{'='*80}")
    print(f"ESTATÍSTICAS - ISP {isp_id}")
    print(f"{'='*80}")
    print(f"Nós da ISP (total): {len(isp_nodes_set)}")
    print(f"Nós após remover desastre: {len(isp_nodes_active)}")
    print(f"Links da ISP: {isp_graph.number_of_edges()}")
    print(f"Conectada: {'Sim' if is_connected else 'Não'}")
    

In [13]:
# ============================================================================
# PATH EXPLORER: Interactive Source-Destination Path Viewer
# ============================================================================

def plot_paths_on_topology(ax, topology_graph, paths_info, source, destination, disaster_node=None, seed=7):
    """
    Plot paths on topology with directional arrows (thickness by frequency).
    
    Args:
        ax: Matplotlib axis
        topology_graph: NetworkX graph
        paths_info: List of path dictionaries with 'caminho', 'distancia', etc.
        source: Source node
        destination: Destination node
        disaster_node: Optional disaster node to highlight
        seed: Layout seed
    """
    if not paths_info:
        ax.text(0.5, 0.5, 'Nenhum caminho encontrado', 
                ha='center', va='center', fontsize=14, transform=ax.transAxes)
        ax.set_title('Sem Caminhos Disponíveis', fontsize=16, fontweight='bold')
        ax.axis('off')
        return
    
    # Count edge frequency in paths (for thickness)
    edge_frequency = defaultdict(int)
    for path_info in paths_info:
        path = path_info['caminho']
        for i in range(len(path) - 1):
            edge = (path[i], path[i+1])
            edge_frequency[edge] += 1
    
    max_freq = max(edge_frequency.values()) if edge_frequency else 1
    
    # Layout
    pos = nx.spring_layout(topology_graph, seed=seed)
    
    ax.set_title(f'Caminhos: {source} → {destination}', fontsize=16, fontweight='bold')
    
    # LAYER 1: Background nodes and edges
    all_nodes = list(topology_graph.nodes())
    nx.draw_networkx_nodes(
        topology_graph, pos,
        nodelist=all_nodes,
        node_color='lightgray',
        node_size=500,
        edgecolors='gray',
        linewidths=1,
        alpha=0.4,
        ax=ax
    )
    
    all_edges = list(topology_graph.edges())
    nx.draw_networkx_edges(
        topology_graph, pos,
        edgelist=all_edges,
        edge_color='lightgray',
        width=1,
        alpha=0.3,
        arrows=False,
        ax=ax
    )
    
    # LAYER 2: Path edges (with directional arrows, colored by rank)
    # Use colormap for path ranking
    cmap = plt.colormaps['viridis']
    colors = [cmap(i / len(paths_info)) for i in range(len(paths_info))]
    
    for path_idx, path_info in enumerate(paths_info):
        path = path_info['caminho']
        color = colors[path_idx]
        
        for i in range(len(path) - 1):
            edge = (path[i], path[i+1])
            freq = edge_frequency[edge]
            
            # Width based on frequency (2-10 range)
            width = 2 + (freq / max_freq) * 8
            
            # Draw directed edge
            nx.draw_networkx_edges(
                topology_graph, pos,
                edgelist=[edge],
                edge_color=[color],
                width=width,
                alpha=0.7,
                arrows=True,
                arrowsize=20,
                arrowstyle='->',
                connectionstyle='arc3,rad=0.1',
                ax=ax
            )
    
    # LAYER 3: Highlight source and destination
    nx.draw_networkx_nodes(
        topology_graph, pos,
        nodelist=[source],
        node_color='green',
        node_size=1000,
        edgecolors='darkgreen',
        linewidths=3,
        ax=ax,
        label='Origem'
    )
    
    nx.draw_networkx_nodes(
        topology_graph, pos,
        nodelist=[destination],
        node_color='blue',
        node_size=1000,
        edgecolors='darkblue',
        linewidths=3,
        ax=ax,
        label='Destino'
    )
    
    # LAYER 4: Disaster node if present
    if disaster_node is not None:
        nx.draw_networkx_nodes(
            topology_graph, pos,
            nodelist=[disaster_node],
            node_color='red',
            node_size=900,
            node_shape='X',
            edgecolors='darkred',
            linewidths=3,
            alpha=0.8,
            ax=ax,
            label='Desastre'
        )
    
    # Node labels
    nx.draw_networkx_labels(
        topology_graph, pos,
        font_size=9,
        font_weight='bold',
        ax=ax
    )
    
    ax.legend(loc='upper left', fontsize=10)
    ax.axis('off')


def plot_ranked_paths(ax, paths_info, source, destination):
    """
    Display ranked list of paths on the right side.
    
    Args:
        ax: Matplotlib axis
        paths_info: List of path dictionaries
        source: Source node
        destination: Destination node
    """
    if not paths_info:
        ax.text(0.5, 0.5, 'Nenhum caminho encontrado', 
                ha='center', va='center', fontsize=12, transform=ax.transAxes)
        ax.set_title('Caminhos Ranqueados', fontsize=16, fontweight='bold')
        ax.axis('off')
        return
    
    ax.set_title(f'Caminhos Ranqueados ({len(paths_info)} encontrados)', 
                 fontsize=16, fontweight='bold')
    
    # Prepare data
    path_labels = [f"#{i+1}" for i in range(len(paths_info))]
    distances = [p['distancia'] for p in paths_info]
    hops = [len(p['caminho']) - 1 for p in paths_info]
    
    # Colors matching the topology plot
    cmap = plt.colormaps['viridis']
    colors = [cmap(i / len(paths_info)) for i in range(len(paths_info))]
    
    y_pos = range(len(paths_info))
    
    # Create horizontal bar chart
    bars = ax.barh(y_pos, distances, color=colors, edgecolor='black', linewidth=1.5)
    
    ax.set_yticks(y_pos)
    ax.set_yticklabels(path_labels, fontsize=10)
    ax.set_xlabel('Distância (km)', fontsize=12, fontweight='bold')
    ax.set_ylabel('Caminho', fontsize=12, fontweight='bold')
    ax.invert_yaxis()  # First path at top
    ax.grid(axis='x', alpha=0.3, linestyle='--')
    
    # Add distance and hop labels
    for i, (dist, hop, path_info) in enumerate(zip(distances, hops, paths_info)):
        # Distance label at end of bar
        ax.text(dist + max(distances) * 0.02, i, f'{dist:.1f} km', 
                va='center', fontsize=9, fontweight='bold')
        
        # Hop count and path on the left
        path_str = ' → '.join(map(str, path_info['caminho']))
        ax.text(-max(distances) * 0.02, i, f'[{hop} hops]  {path_str}', 
                va='center', ha='right', fontsize=8, style='italic')
    
    # Adjust limits to fit labels
    ax.set_xlim(-max(distances) * 0.5, max(distances) * 1.15)


In [14]:
# ============================================================================
# UNIFIED GUI: ISP Topology + Path Explorer (Combined)
# ============================================================================

import ipywidgets as widgets
from IPython.display import display, clear_output
from itertools import islice

def create_unified_isp_gui(
    lista_de_isps,
    topology,
    disaster_node,
    weights_by_link_by_isp
):
    """
    Create unified GUI for ISP visualization and path exploration.
    Paths are computed using ISP-specific weighted graphs.
    
    Args:
        lista_de_isps: List of ISP objects
        topology: Topology object with .topology (NetworkX graph)
        disaster_node: Node affected by disaster
        weights_by_link_by_isp: Computed weights for all ISPs
    """
    # Create output widgets
    output_topology = widgets.Output()  # For ISP topology view
    output_paths = widgets.Output()     # For path explorer view
    output_usage = widgets.Output()     # For ISP usage analysis view
    output_migration = widgets.Output() # For migration analysis view
    output_criticality = widgets.Output() # For link criticality view
    output_global_comparison = widgets.Output() # For global comparison view
    
    # ========== Helper Functions ==========
    
    def calculate_path_isp_sharing(path, lista_de_isps):
        """Calculate average number of ISPs sharing links in a path."""
        link_isp_counts = []
        for i in range(len(path) - 1):
            link = tuple(sorted([path[i], path[i+1]]))
            isp_count = sum(1 for isp in lista_de_isps 
                           if link in [tuple(sorted(e)) for e in isp.edges])
            link_isp_counts.append(isp_count)
        return sum(link_isp_counts) / len(link_isp_counts) if link_isp_counts else 0
    
    # ========== ISP Selection ==========
    isp_options = [(f"ISP {isp.isp_id}", isp.isp_id) for isp in lista_de_isps]
    isp_dropdown = widgets.Dropdown(
        options=isp_options,
        value=lista_de_isps[0].isp_id,
        description='ISP:',
        style={'description_width': '60px'},
        layout=widgets.Layout(width='200px')
    )
    
    # ========== View Mode Selection ==========
    view_mode = widgets.ToggleButtons(
        options=['ISP Topology', 'Path Explorer', 'ISP Usage Analysis', 'Migration Analysis', 'Link Criticality', 'Global Comparison'],
        value='ISP Topology',
        description='View:',
        style={'description_width': '60px'},
        button_style='info',
        layout=widgets.Layout(width='1000px')
    )
    
    # Helper function to get ISP nodes
    def get_isp_nodes(isp_id):
        """Get nodes allocated to a specific ISP."""
        isp = lista_de_isps[isp_id]
        return sorted(isp.nodes)
    
    # ========== Path Explorer Controls ==========
    # Initialize with first ISP's nodes
    initial_isp_nodes = get_isp_nodes(lista_de_isps[0].isp_id)
    
    source_dropdown = widgets.Dropdown(
        options=initial_isp_nodes,
        value=initial_isp_nodes[0] if initial_isp_nodes else None,
        description='Origem:',
        style={'description_width': '60px'},
        layout=widgets.Layout(width='200px')
    )
    
    dest_dropdown = widgets.Dropdown(
        options=initial_isp_nodes,
        value=initial_isp_nodes[1] if len(initial_isp_nodes) > 1 else initial_isp_nodes[0],
        description='Destino:',
        style={'description_width': '60px'},
        layout=widgets.Layout(width='200px')
    )
    
    k_slider = widgets.IntSlider(
        value=5,
        min=1,
        max=10,
        step=1,
        description='K Paths:',
        style={'description_width': '60px'},
        layout=widgets.Layout(width='300px')
    )
    
    # ========== Disaster Mode Toggle (for Path Explorer) ==========
    disaster_mode_checkbox = widgets.Checkbox(
        value=False,
        description='Disaster Mode (Weighted + Remove Disaster Node)',
        style={'description_width': 'initial'},
        layout=widgets.Layout(width='450px')
    )
    
    # ========== Show Original Paths Toggle (for Disaster Mode) ==========
    show_original_paths_checkbox = widgets.Checkbox(
        value=False,
        description='Show Original Paths (α=β=γ=0)',
        style={'description_width': 'initial'},
        layout=widgets.Layout(width='300px')
    )
    
    # ========== Disaster View Toggle (for ISP Topology) ==========
    show_disaster_view_checkbox = widgets.Checkbox(
        value=True,
        description='Show Disaster View (Mark disaster node & show impact)',
        style={'description_width': 'initial'},
        layout=widgets.Layout(width='450px')
    )
    
    # ========== Link Selector (for ISP Usage Analysis) ==========
    link_dropdown = widgets.Dropdown(
        options=[],
        description='Select Link:',
        style={'description_width': '100px'},
        layout=widgets.Layout(width='400px')
    )
    
    # ========== Migration Paths Slider (for Migration Analysis) ==========
    migration_paths_slider = widgets.IntSlider(
        value=10,
        min=1,
        max=20,
        step=1,
        description='Paths per ISP:',
        style={'description_width': '100px'},
        layout=widgets.Layout(width='400px')
    )
    
    # ========== Migration Weight K-Paths Slider ==========
    migration_weight_k_slider = widgets.IntSlider(
        value=5,
        min=1,
        max=10,
        step=1,
        description='Migration K:',
        style={'description_width': '100px'},
        layout=widgets.Layout(width='400px')
    )
    
    # ========== ISP Toggle Checkboxes (for Migration Analysis) ==========
    isp_checkboxes = {}
    for isp in lista_de_isps:
        if hasattr(isp, 'datacenter') and isp.datacenter is not None:
            isp_checkboxes[isp.isp_id] = widgets.Checkbox(
                value=True,
                description=f'ISP {isp.isp_id}',
                style={'description_width': 'initial'},
                layout=widgets.Layout(width='auto', min_width='100px')
            )
    
    # ========== Link Criticality Controls ==========
    
    # Disaster mode toggle for criticality
    criticality_disaster_mode_checkbox = widgets.Checkbox(
        value=False,
        description='Disaster Mode (remove disaster node)',
        style={'description_width': 'initial'},
        layout=widgets.Layout(width='300px')
    )
    
    # ========== Global Comparison Controls ==========
    
    # Paths per node pair slider
    global_k_slider = widgets.IntSlider(
        value=5,
        min=1,
        max=10,
        step=1,
        description='Paths per pair:',
        style={'description_width': '100px'},
        layout=widgets.Layout(width='400px')
    )
    
    # ISP checkboxes for global comparison (similar to migration analysis)
    global_isp_checkboxes = {}
    for isp in lista_de_isps:
        global_isp_checkboxes[isp.isp_id] = widgets.Checkbox(
            value=True,
            description=f'ISP {isp.isp_id}',
            style={'description_width': 'initial'},
            layout=widgets.Layout(width='auto', min_width='100px')
        )
    
    # ========== Weight Parameter Sliders ==========
    alfa_slider = widgets.FloatSlider(
        value=0.2,
        min=0.0,
        max=0.5,
        step=0.05,
        description='α (ISP):',
        style={'description_width': '80px'},
        layout=widgets.Layout(width='300px'),
        readout_format='.2f'
    )
    
    beta_slider = widgets.FloatSlider(
        value=0.2,
        min=0.0,
        max=0.5,
        step=0.05,
        description='β (Migr):',
        style={'description_width': '80px'},
        layout=widgets.Layout(width='300px'),
        readout_format='.2f'
    )
    
    gamma_slider = widgets.FloatSlider(
        value=0.4,
        min=0.0,
        max=0.8,
        step=0.05,
        description='γ (Crit):',
        style={'description_width': '80px'},
        layout=widgets.Layout(width='300px'),
        readout_format='.2f'
    )
    
    recalc_button = widgets.Button(
        description='🔄 Recalcular Pesos',
        button_style='success',
        layout=widgets.Layout(width='180px')
    )
    
    weights_info_label = widgets.HTML(
        value="<p style='font-size: 10px; color: #666;'>"
              "α: ISP Usage | β: Migration | γ: Link Criticality</p>",
        layout=widgets.Layout(margin='0 0 5px 0')
    )
    
    # ========== Info and Status Labels ==========
    _info_label = widgets.HTML(
        value="<h3>🌐 ISP Analysis & Path Explorer</h3>"
              "<p><b>ISP Topology:</b> View ISP-specific weights<br>"
              "• <i>Disaster View ON</i>: Shows disaster node ❌ and resulting topology<br>"
              "• <i>Disaster View OFF</i>: Shows normal ISP topology<br>"
              "<b>Path Explorer:</b> Paths within ISP subnet only<br>"
              "• <i>Normal Mode</i>: Unweighted paths (physical distance)<br>"
              "• <i>Disaster Mode</i>: Weighted paths + Disaster node removed<br>"
              "<b>ISP Usage Analysis:</b> Visualize link sharing across ISPs<br>"
              "• <i>Left</i>: Network with links colored by # of ISPs sharing them<br>"
              "• <i>Right</i>: Details about selected link and which ISPs use it<br>"
              "<b>Migration Analysis:</b> Visualize datacenter migrations<br>"
              "• <i>Left</i>: Migration paths with directional arrows, thickness = usage<br>"
              "• <i>Right</i>: Statistics about migrations and most used links<br>"
              "• <i>Paths per ISP slider</i>: Control number of paths shown per ISP (1-20)<br>"
              "• <i>Show ISPs toggles</i>: Enable/disable individual ISPs (default: all enabled)<br>"
              "<b>Link Criticality:</b> Network robustness & bridge detection<br>"
              "• <i>Disaster Mode toggle</i>: Analyze with/without disaster node<br>"
              "• <i>Left</i>: Bridges colored by γ weight (🟡→🟠→🔴), non-bridges grey<br>"
              "• <i>Color intensity</i>: Reflects actual gamma penalty (varies with γ)<br>"
              "• <i>Right (default)</i>: Bridges ranked by # of affected ISPs<br>"
              "• <i>Select Link</i>: Dropdown shows only bridges, sorted by impact<br>"
              "• <i>Bridge Detection</i>: At ISP subnet level (not global)<br>"
              "<b>Global Comparison:</b> Compare all node pairs across ISPs<br>"
              "• <i>Shows table</i>: Weighted vs unweighted (α=β=γ=0) disaster paths<br>"
              "• <i>Select ISPs</i>: Toggle which ISPs to include in comparison<br>"
              "• <i>Paths per pair</i>: Control number of paths analyzed per node pair<br>"
              "• <i>Displays</i>: Δ Hops, Δ Modulation, Δ ISP Sharing, Δ Weight (%)<br>"
              "<b>Weight Parameters:</b><br>"
              "• <b>Migration K slider</b>: Control paths for migration weight calculation (1-10)<br>"
              "• <i>Higher K</i>: More alternative migration paths considered<br>"
              "• <b>Adjust α, β, γ, Migration K</b> and click 🔄 to recalculate weights dynamically!</p>",
        layout=widgets.Layout(margin='0 0 15px 0')
    )
    
    status_label = widgets.HTML(
        value="",
        layout=widgets.Layout(margin='10px 0')
    )
    
    # Container for path controls (hidden initially)
    path_controls_row1 = widgets.HBox(
        [source_dropdown, dest_dropdown, k_slider],
        layout=widgets.Layout(display='none', margin='5px 0')
    )
    
    path_controls_row2 = widgets.HBox(
        [disaster_mode_checkbox, show_original_paths_checkbox],
        layout=widgets.Layout(display='none', margin='5px 0')
    )
    
    # Container for ISP topology controls (hidden initially)
    topology_controls_row = widgets.HBox(
        [show_disaster_view_checkbox],
        layout=widgets.Layout(display='flex', margin='5px 0')
    )
    
    # Container for ISP usage analysis controls (hidden initially)
    usage_controls_row = widgets.HBox(
        [link_dropdown],
        layout=widgets.Layout(display='none', margin='5px 0')
    )
    
    # Container for migration analysis controls (hidden initially)
    # Organize ISP checkboxes into rows of 3
    checkbox_list = list(isp_checkboxes.values())
    checkbox_rows = []
    for i in range(0, len(checkbox_list), 3):
        row = widgets.HBox(
            checkbox_list[i:i+3],
            layout=widgets.Layout(margin='2px 0')
        )
        checkbox_rows.append(row)
    
    # Stack checkbox rows vertically
    isp_checkboxes_box = widgets.VBox(
        checkbox_rows,
        layout=widgets.Layout(margin='0 0 0 20px')
    )
    
    # Label for ISP toggles
    isp_toggles_label = widgets.Label(
        'Show ISPs:',
        layout=widgets.Layout(width='80px', margin='0 0 5px 0')
    )
    
    # First row: slider
    migration_controls_row1 = widgets.HBox(
        [migration_paths_slider],
        layout=widgets.Layout(margin='5px 0')
    )
    
    # Second row: ISP toggles
    migration_controls_row2 = widgets.HBox(
        [isp_toggles_label, isp_checkboxes_box],
        layout=widgets.Layout(margin='5px 0')
    )
    
    # Stack both rows vertically
    migration_controls_row = widgets.VBox(
        [migration_controls_row1, migration_controls_row2],
        layout=widgets.Layout(display='none', margin='5px 0')
    )
    
    # Container for link criticality controls (hidden initially)
    criticality_controls_row1 = widgets.HBox(
        [criticality_disaster_mode_checkbox],
        layout=widgets.Layout(margin='5px 0')
    )
    
    criticality_controls_row = widgets.VBox(
        [criticality_controls_row1],
        layout=widgets.Layout(display='none', margin='5px 0')
    )
    
    # ========== Global Comparison Controls Layout ==========
    global_comparison_controls_row1 = widgets.HBox(
        [global_k_slider],
        layout=widgets.Layout(margin='5px 0')
    )
    
    global_comparison_controls_row2 = widgets.HBox(
        list(global_isp_checkboxes.values()),
        layout=widgets.Layout(margin='5px 0')
    )
    
    global_comparison_controls_row = widgets.VBox(
        [global_comparison_controls_row1, global_comparison_controls_row2],
        layout=widgets.Layout(display='none', margin='5px 0')
    )
    
    # Container for weight parameter controls
    weight_params_row = widgets.HBox(
        [alfa_slider, beta_slider, gamma_slider, migration_weight_k_slider, recalc_button],
        layout=widgets.Layout(margin='10px 0')
    )
    
    # Store current weights (initialize with visualization-based migration weights)
    initial_migration_weights = calculate_migration_weights_for_visualization(
        lista_de_isps, 
        topology.topology, 
        disaster_node,
        num_paths=5,
        beta=beta
    )
    
    initial_combined_weights = calculate_weights_by_isps(
        lista_de_isps,
        isp_usage_weights,
        initial_migration_weights,
        link_criticality_weights
    )
    
    current_weights = {
        'isp_usage': isp_usage_weights,
        'migration': initial_migration_weights,
        'link_criticality': link_criticality_weights,
        'combined': initial_combined_weights
    }
    
    # ========== Weight Recalculation Function ==========
    
    def recalculate_weights(button=None):
        """Recalculate all weights with new alpha, beta, gamma values."""
        nonlocal current_weights
        
        alfa = alfa_slider.value
        beta = beta_slider.value
        gamma = gamma_slider.value
        migration_k = migration_weight_k_slider.value
        
        status_label.value = f"<p style='color: blue;'>🔄 Recalculando pesos com α={alfa:.2f}, β={beta:.2f}, γ={gamma:.2f}, Migration K={migration_k}...</p>"
        
        try:
            # Recalculate ISP Usage Weights
            isp_usage_new = calculate_isp_usage_weights(lista_de_isps, alfa)
            
            # Recalculate Migration Weights with K paths
            migration_new = calculate_migration_weights_for_visualization(
                lista_de_isps, 
                topology.topology, 
                disaster_node,
                num_paths=migration_k,
                beta=beta
            )
            
            # Recalculate Link Criticality
            link_criticality_new = calculate_link_criticality(topology, disaster_node, gamma)
            
            # Combine all weights
            combined_new = calculate_weights_by_isps(
                lista_de_isps,
                isp_usage_new,
                migration_new,
                link_criticality_new
            )
            
            # Update current weights
            current_weights['isp_usage'] = isp_usage_new
            current_weights['migration'] = migration_new
            current_weights['link_criticality'] = link_criticality_new
            current_weights['combined'] = combined_new
            
            status_label.value = f"<p style='color: green;'>✅ Pesos recalculados com sucesso! α={alfa:.2f}, β={beta:.2f}, γ={gamma:.2f}, Migration K={migration_k}</p>"
            
            # Refresh current view
            if view_mode.value == 'ISP Topology':
                update_isp_topology()
            elif view_mode.value == 'Link Criticality':
                update_link_criticality()
            elif view_mode.value == 'Global Comparison':
                update_global_comparison()
            else:
                update_path_explorer()
                
        except Exception as e:
            status_label.value = f"<p style='color: red;'>❌ Erro ao recalcular pesos: {str(e)}</p>"
    
    # ========== Populate Link Dropdown ==========
    def populate_link_dropdown():
        """Populate link dropdown with all edges from topology."""
        all_edges = list(topology.topology.edges())
        # Normalize to avoid duplicates
        unique_edges = set()
        for u, v in all_edges:
            unique_edges.add(tuple(sorted([u, v])))
        
        # Sort by node IDs
        sorted_edges = sorted(unique_edges)
        link_options = [(f"{u} ↔ {v}", (u, v)) for u, v in sorted_edges]
        link_dropdown.options = link_options
        if link_options:
            link_dropdown.value = link_options[0][1]
    
    # Initialize link dropdown
    populate_link_dropdown()
    
    # ========== Global Comparison Functions ==========
    
    def calculate_global_comparison_data(
        lista_de_isps,
        topology,
        disaster_node,
        current_weights,
        k_paths,
        active_isps
    ):
        """
        Calculate comparison statistics for all node pairs in selected ISPs.
        
        Returns:
            List of dictionaries with:
        - isp_id
        - source, destination
        - num_paths (actual number found)
        - avg_hops_unweighted, avg_hops_weighted, hops_change_pct
        - avg_modulation_unweighted, avg_modulation_weighted, modulation_change_pct
        - avg_isp_sharing_unweighted, avg_isp_sharing_weighted, isp_sharing_change_pct
        - avg_weight_unweighted, avg_weight_weighted, weight_change_pct
        """
        from simulador.core.path_manager import PathManager
        
        comparison_data = []
        
        for isp_id in active_isps:
            isp = lista_de_isps[isp_id]
            
            # Get ISP nodes excluding disaster node
            isp_nodes = [n for n in isp.nodes if n != disaster_node]
            
            if len(isp_nodes) < 2:
                continue
            
            # Build ISP subgraph without disaster node
            isp_subgraph = topology.topology.subgraph(isp_nodes).copy()
            for edge in isp.edges:
                if (edge[0] in isp_nodes and edge[1] in isp_nodes 
                    and not isp_subgraph.has_edge(edge[0], edge[1])
                    and topology.topology.has_edge(edge[0], edge[1])):
                    edge_data = topology.topology[edge[0]][edge[1]].copy()
                    isp_subgraph.add_edge(edge[0], edge[1], **edge_data)
            
            # Create weighted graph
            weighted_graph = create_weighted_graph(
                isp_subgraph,
                isp_id,
                current_weights['combined']
            )
            
            # Calculate paths for all node pairs
            for i, source in enumerate(isp_nodes):
                for j, destination in enumerate(isp_nodes):
                    if i >= j:  # Avoid duplicates and self-loops
                        continue
                    
                    try:
                        # Calculate unweighted paths
                        unweighted_paths = list(islice(
                            nx.shortest_simple_paths(isp_subgraph, source, destination, weight='weight'),
                            k_paths
                        ))
                        
                        # Calculate weighted paths
                        weighted_paths = list(islice(
                            nx.shortest_simple_paths(weighted_graph, source, destination, weight='weight'),
                            k_paths
                        ))
                        
                        if not unweighted_paths or not weighted_paths:
                            continue
                        
                        # Calculate statistics for unweighted paths
                        unweighted_hops = [len(p) - 1 for p in unweighted_paths]
                        unweighted_distances = [PathManager.calculate_path_distance(isp_subgraph, p) for p in unweighted_paths]
                        unweighted_modulations = [PathManager.calculate_modulation_factor(d) for d in unweighted_distances]
                        unweighted_isp_sharing = [calculate_path_isp_sharing(p, lista_de_isps) for p in unweighted_paths]
                        
                        # Calculate statistics for weighted paths
                        weighted_hops = [len(p) - 1 for p in weighted_paths]
                        weighted_distances = [PathManager.calculate_path_distance(isp_subgraph, p) for p in weighted_paths]
                        weighted_modulations = [PathManager.calculate_modulation_factor(d) for d in weighted_distances]
                        weighted_isp_sharing = [calculate_path_isp_sharing(p, lista_de_isps) for p in weighted_paths]
                        weighted_weights = [
                            sum(weighted_graph[p[k]][p[k+1]]['weight'] for k in range(len(p) - 1))
                            for p in weighted_paths
                        ]
                        
                        # Calculate averages
                        avg_hops_unweighted = sum(unweighted_hops) / len(unweighted_hops)
                        avg_hops_weighted = sum(weighted_hops) / len(weighted_hops)
                        hops_change_pct = ((avg_hops_weighted - avg_hops_unweighted) / avg_hops_unweighted * 100) if avg_hops_unweighted > 0 else 0
                        
                        avg_modulation_unweighted = sum(unweighted_modulations) / len(unweighted_modulations)
                        avg_modulation_weighted = sum(weighted_modulations) / len(weighted_modulations)
                        modulation_change_pct = ((avg_modulation_weighted - avg_modulation_unweighted) / avg_modulation_unweighted * 100) if avg_modulation_unweighted > 0 else 0
                        
                        avg_isp_sharing_unweighted = sum(unweighted_isp_sharing) / len(unweighted_isp_sharing)
                        avg_isp_sharing_weighted = sum(weighted_isp_sharing) / len(weighted_isp_sharing)
                        isp_sharing_change_pct = ((avg_isp_sharing_weighted - avg_isp_sharing_unweighted) / avg_isp_sharing_unweighted * 100) if avg_isp_sharing_unweighted > 0 else 0
                        
                        avg_weight_unweighted = sum(unweighted_distances) / len(unweighted_distances)
                        avg_weight_weighted = sum(weighted_weights) / len(weighted_weights)
                        weight_change_pct = ((avg_weight_weighted - avg_weight_unweighted) / avg_weight_unweighted * 100) if avg_weight_unweighted > 0 else 0
                        
                        comparison_data.append({
                            'isp_id': isp_id,
                            'source': source,
                            'destination': destination,
                            'num_paths': len(unweighted_paths),
                            'avg_hops_unweighted': avg_hops_unweighted,
                            'avg_hops_weighted': avg_hops_weighted,
                            'hops_change_pct': hops_change_pct,
                            'avg_modulation_unweighted': avg_modulation_unweighted,
                            'avg_modulation_weighted': avg_modulation_weighted,
                            'modulation_change_pct': modulation_change_pct,
                            'avg_isp_sharing_unweighted': avg_isp_sharing_unweighted,
                            'avg_isp_sharing_weighted': avg_isp_sharing_weighted,
                            'isp_sharing_change_pct': isp_sharing_change_pct,
                            'avg_weight_unweighted': avg_weight_unweighted,
                            'avg_weight_weighted': avg_weight_weighted,
                            'weight_change_pct': weight_change_pct
                        })
                    
                    except nx.NetworkXNoPath:
                        continue
        
        return comparison_data
    
    def visualize_global_comparison(comparison_data, k_paths):
        """
        Visualize global comparison data as aggregated statistics.
        """
        if not comparison_data:
            fig, ax = plt.subplots(figsize=(12, 6))
            ax.text(0.5, 0.5, 'No comparison data available',
                     ha='center', va='center', fontsize=14, transform=ax.transAxes)
            ax.set_title('Global Comparison - No Data', fontsize=16, fontweight='bold')
            ax.axis('off')
            plt.tight_layout()
            plt.show()
            return
        
        # Calculate overall statistics
        total_pairs = len(comparison_data)
        total_isps = len(set(row['isp_id'] for row in comparison_data))
        
        # Calculate weighted averages (by number of paths per pair)
        total_paths = sum(row['num_paths'] for row in comparison_data)
        
        # Calculate overall averages
        avg_hops_unweighted = sum(row['avg_hops_unweighted'] * row['num_paths'] for row in comparison_data) / total_paths
        avg_hops_weighted = sum(row['avg_hops_weighted'] * row['num_paths'] for row in comparison_data) / total_paths
        hops_change_pct = ((avg_hops_weighted - avg_hops_unweighted) / avg_hops_unweighted * 100) if avg_hops_unweighted > 0 else 0
        
        avg_modulation_unweighted = sum(row['avg_modulation_unweighted'] * row['num_paths'] for row in comparison_data) / total_paths
        avg_modulation_weighted = sum(row['avg_modulation_weighted'] * row['num_paths'] for row in comparison_data) / total_paths
        modulation_change_pct = ((avg_modulation_weighted - avg_modulation_unweighted) / avg_modulation_unweighted * 100) if avg_modulation_unweighted > 0 else 0
        
        avg_isp_sharing_unweighted = sum(row['avg_isp_sharing_unweighted'] * row['num_paths'] for row in comparison_data) / total_paths
        avg_isp_sharing_weighted = sum(row['avg_isp_sharing_weighted'] * row['num_paths'] for row in comparison_data) / total_paths
        isp_sharing_change_pct = ((avg_isp_sharing_weighted - avg_isp_sharing_unweighted) / avg_isp_sharing_unweighted * 100) if avg_isp_sharing_unweighted > 0 else 0
        
        avg_weight_unweighted = sum(row['avg_weight_unweighted'] * row['num_paths'] for row in comparison_data) / total_paths
        avg_weight_weighted = sum(row['avg_weight_weighted'] * row['num_paths'] for row in comparison_data) / total_paths
        weight_change_pct = ((avg_weight_weighted - avg_weight_unweighted) / avg_weight_unweighted * 100) if avg_weight_unweighted > 0 else 0
        
        # Create figure
        fig, ax = plt.subplots(figsize=(14, 8))
        
        # Create summary table data
        metrics = [
            ('Average Hops', avg_hops_unweighted, avg_hops_weighted, hops_change_pct),
            ('Average Modulation', avg_modulation_unweighted, avg_modulation_weighted, modulation_change_pct),
            ('Average ISP Sharing', avg_isp_sharing_unweighted, avg_isp_sharing_weighted, isp_sharing_change_pct),
            ('Average Weight', avg_weight_unweighted, avg_weight_weighted, weight_change_pct)
        ]
        
        # Create table
        table_data = []
        for metric_name, unweighted_val, weighted_val, change_pct in metrics:
            table_data.append([
                metric_name,
                f"{unweighted_val:.3f}",
                f"{weighted_val:.3f}",
                f"{change_pct:+.1f}%"
            ])
        
        # Table headers
        headers = ['Metric', 'Unweighted (α=β=γ=0)', 'Weighted (Current)', 'Change (%)']
        
        # Create table
        table = ax.table(
            cellText=table_data,
            colLabels=headers,
            cellLoc='center',
            loc='center',
            bbox=[0, 0, 1, 0.7]
        )
        
        # Style the table
        table.auto_set_font_size(False)
        table.set_fontsize(12)
        table.scale(1, 2)
        
        # Color code the change column
        for i in range(len(table_data)):
            cell = table[(i+1, 3)]  # Change column
            change_str = table_data[i][3]
            try:
                change_val = float(change_str.replace('%', '').replace('+', ''))
                if change_val < 0:
                    cell.set_facecolor('#d4edda')  # Light green
                elif change_val > 0:
                    cell.set_facecolor('#f8d7da')  # Light red
                else:
                    cell.set_facecolor('#ffffff')  # White
            except:
                cell.set_facecolor('#ffffff')
        
        # Style header
        for j in range(len(headers)):
            table[(0, j)].set_facecolor('#343a40')
            table[(0, j)].set_text_props(weight='bold', color='white')
        
        ax.set_title(f'Global Comparison: Weighted vs Unweighted Disaster Paths (k={k_paths})\n'
                     f'Total Pairs: {total_pairs} | ISPs: {total_isps} | Total Paths: {total_paths}',
                      fontsize=16, fontweight='bold', pad=20)
        ax.axis('off')
        
        plt.tight_layout()
        plt.show()
    
    # ========== Update Functions ==========
    
    def update_isp_topology():
        """Show ISP topology with weights."""
        with output_topology:
            clear_output(wait=True)
            
            isp_id = isp_dropdown.value
            isp = lista_de_isps[isp_id]
            show_disaster = show_disaster_view_checkbox.value
            
            mode_str = "Disaster View (with impact)" if show_disaster else "Normal View"
            
            visualize_isp_topology_with_weights(
                isp,
                topology.topology,
                disaster_node,
                isp_id,
                current_weights['combined'],
                remove_disaster_node=show_disaster
            )
    
    def update_isp_node_dropdowns():
        """Update source/destination dropdowns based on selected ISP."""
        isp_id = isp_dropdown.value
        isp_nodes = get_isp_nodes(isp_id)
        
        # Store current selections
        current_source = source_dropdown.value
        current_dest = dest_dropdown.value
        
        # Update options
        source_dropdown.options = isp_nodes
        dest_dropdown.options = isp_nodes
        
        # Restore selections if they're still valid, otherwise reset
        if current_source in isp_nodes:
            source_dropdown.value = current_source
        else:
            source_dropdown.value = isp_nodes[0] if isp_nodes else None
        
        if current_dest in isp_nodes:
            dest_dropdown.value = current_dest
        else:
            dest_dropdown.value = isp_nodes[1] if len(isp_nodes) > 1 else isp_nodes[0]
    
    def update_isp_usage_analysis():
        """Show ISP usage analysis view."""
        with output_usage:
            clear_output(wait=True)
            
            selected_link = link_dropdown.value
            
            visualize_isp_usage_analysis(
                topology.topology,
                lista_de_isps,
                selected_link=selected_link
            )
    
    def update_migration_analysis():
        """Show migration analysis view."""
        with output_migration:
            clear_output(wait=True)
            
            max_paths = migration_paths_slider.value
            
            # Collect active ISPs from checkboxes
            active_isps = set()
            for isp_id, checkbox in isp_checkboxes.items():
                if checkbox.value:
                    active_isps.add(isp_id)
            
            visualize_migration_analysis(
                topology.topology,
                lista_de_isps,
                disaster_node,
                max_paths_per_isp=max_paths,
                active_isps=active_isps
            )
    
    def update_link_criticality():
        """Show link criticality analysis view."""
        with output_criticality:
            clear_output(wait=True)
            
            disaster_mode = criticality_disaster_mode_checkbox.value
            
            mode_text = "Disaster Mode" if disaster_mode else "Normal Mode"
            
            result = visualize_link_criticality_analysis(
                topology.topology,
                lista_de_isps,
                disaster_node,
                current_weights,
                disaster_mode=disaster_mode
            )
            
            # Store result for dropdown updates
            return result
    
    def update_global_comparison():
        """Show global comparison view."""
        with output_global_comparison:
            clear_output(wait=True)
            
            k = global_k_slider.value
            
            # Collect active ISPs from checkboxes
            active_isps = set()
            for isp_id, checkbox in global_isp_checkboxes.items():
                if checkbox.value:
                    active_isps.add(isp_id)
            
            if not active_isps:
                fig, ax = plt.subplots(figsize=(12, 6))
                ax.text(0.5, 0.5, 'No ISPs selected for comparison',
                         ha='center', va='center', fontsize=14, transform=ax.transAxes)
                ax.set_title('Global Comparison - No ISPs Selected', fontsize=16, fontweight='bold')
                ax.axis('off')
                plt.tight_layout()
                plt.show()
                return
            
            status_label.value = f"<p style='color: blue;'>🔄 Calculating global comparison for {len(active_isps)} ISP(s) with k={k} paths per pair...</p>"
            
            try:
                # Calculate comparison data
                comparison_data = calculate_global_comparison_data(
                    lista_de_isps,
                    topology,
                    disaster_node,
                    current_weights,
                    k,
                    active_isps
                )
                
                # Visualize
                visualize_global_comparison(comparison_data, k)
                
                status_label.value = f"<p style='color: green;'>✅ Global comparison completed! Analyzed {len(comparison_data)} node pairs across {len(active_isps)} ISP(s).</p>"
                
            except Exception as e:
                status_label.value = f"<p style='color: red;'>❌ Error in global comparison: {str(e)}</p>"
                import traceback
                traceback.print_exc()
    
    def update_path_explorer():
        """Show path explorer with normal or disaster-aware paths."""
        with output_paths:
            clear_output(wait=True)
            
            isp_id = isp_dropdown.value
            isp = lista_de_isps[isp_id]
            source = source_dropdown.value
            destination = dest_dropdown.value
            k = k_slider.value
            disaster_mode = disaster_mode_checkbox.value
            
            # Validation
            if source == destination:
                status_label.value = "<p style='color: orange;'>⚠️ Origem e destino devem ser diferentes!</p>"
                print("⚠️ Origem e destino devem ser diferentes!")
                return
            
            # Validate source and destination are in ISP
            if source not in isp.nodes or destination not in isp.nodes:
                status_label.value = "<p style='color: orange;'>⚠️ Origem e destino devem estar na ISP selecionada!</p>"
                print(f"⚠️ Source {source} or destination {destination} not in ISP {isp_id}")
                return
            
            mode_str = "Disaster-Aware (Weighted)" if disaster_mode else "Normal (Unweighted)"
            status_label.value = f"<p style='color: blue;'>🔄 Calculando {k} caminhos [{mode_str}] para ISP {isp_id}: {source} → {destination}...</p>"
            
            try:
                from simulador.core.path_manager import PathManager
                
                # Build ISP subgraph
                isp_subgraph = topology.topology.subgraph(isp.nodes).copy()
                for edge in isp.edges:
                    if (edge[0] in isp.nodes and edge[1] in isp.nodes 
                        and not isp_subgraph.has_edge(edge[0], edge[1])
                        and topology.topology.has_edge(edge[0], edge[1])):
                        edge_data = topology.topology[edge[0]][edge[1]].copy()
                        isp_subgraph.add_edge(edge[0], edge[1], **edge_data)
                
                if disaster_mode:
                    # === DISASTER MODE: Weighted graph without disaster node ===
                    
                    # Remove disaster node from ISP subgraph
                    isp_subgraph_no_disaster = isp_subgraph.copy()
                    if disaster_node in isp_subgraph_no_disaster.nodes():
                        isp_subgraph_no_disaster.remove_node(disaster_node)
                    
                    # Apply ISP-specific weights to the subgraph
                    weighted_graph = create_weighted_graph(
                        isp_subgraph_no_disaster,
                        isp_id,
                        current_weights['combined']
                    )
                    
                    # Compute k-shortest paths on weighted ISP subgraph without disaster node
                    paths = list(islice(
                        nx.shortest_simple_paths(weighted_graph, source, destination, weight='weight'),
                        k
                    ))
                    
                    # Build paths_info
                    paths_info = []
                    for path in paths:
                        distance = PathManager.calculate_path_distance(isp_subgraph_no_disaster, path)
                        modulation = PathManager.calculate_modulation_factor(distance)
                        weighted_distance = sum(
                            weighted_graph[path[i]][path[i+1]]['weight']
                            for i in range(len(path) - 1)
                        )
                        isp_sharing = calculate_path_isp_sharing(path, lista_de_isps)
                        paths_info.append({
                            'caminho': path,
                            'distancia': distance,
                            'weighted_distance': weighted_distance,
                            'fator_de_modulacao': modulation,
                            'isp_sharing': isp_sharing
                        })
                    
                    # Always calculate original paths (unweighted) for comparison
                    original_paths = list(islice(
                        nx.shortest_simple_paths(isp_subgraph_no_disaster, source, destination, weight='weight'),
                        k
                    ))
                    
                    original_paths_info = []
                    for path in original_paths:
                        distance = PathManager.calculate_path_distance(isp_subgraph_no_disaster, path)
                        modulation = PathManager.calculate_modulation_factor(distance)
                        isp_sharing = calculate_path_isp_sharing(path, lista_de_isps)
                        original_paths_info.append({
                            'caminho': path,
                            'distancia': distance,
                            'weighted_distance': distance,  # Same as real distance in unweighted
                            'fator_de_modulacao': modulation,
                            'isp_sharing': isp_sharing
                        })
                    
                    # Visualization
                    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 10))
                    
                    # Show original paths in left plot if toggle is enabled, otherwise show weighted paths
                    if show_original_paths_checkbox.value:
                        plot_paths_on_topology(ax1, topology.topology, original_paths_info, source, destination, disaster_node)
                        ax1.set_title(f'ISP {isp_id} - Original Paths (α=β=γ=0): {source} → {destination}', 
                                     fontsize=16, fontweight='bold', color='darkgreen')
                    else:
                        plot_paths_on_topology(ax1, topology.topology, paths_info, source, destination, disaster_node)
                        ax1.set_title(f'ISP {isp_id} - Disaster-Aware Paths (ISP Subnet): {source} → {destination}', 
                                     fontsize=16, fontweight='bold', color='darkred')
                    
                    if show_original_paths_checkbox.value:
                        display_paths_comparison(ax2, original_paths_info, source, destination, isp_id, 
                                               original_paths_info=paths_info, 
                                               show_comparison=True)
                    else:
                        display_paths_comparison(ax2, paths_info, source, destination, isp_id, 
                                               original_paths_info=original_paths_info, 
                                               show_comparison=True)
                    
                    plt.tight_layout()
                    plt.show()
                    
                    status_label.value = f"<p style='color: green;'>✅ {len(paths_info)} disaster-aware path(s) found in ISP {isp_id} subnet!</p>"
                else:
                    # === NORMAL MODE: Unweighted, ISP subnet only ===
                    
                    # Compute k-shortest paths on ISP subgraph (unweighted)
                    paths = list(islice(
                        nx.shortest_simple_paths(isp_subgraph, source, destination, weight='weight'),
                        k
                    ))
                    
                    # Build paths_info
                    paths_info = []
                    for path in paths:
                        distance = PathManager.calculate_path_distance(isp_subgraph, path)
                        modulation = PathManager.calculate_modulation_factor(distance)
                        isp_sharing = calculate_path_isp_sharing(path, lista_de_isps)
                        paths_info.append({
                            'caminho': path,
                            'distancia': distance,
                            'weighted_distance': distance,  # Same as real distance in normal mode
                            'fator_de_modulacao': modulation,
                            'isp_sharing': isp_sharing
                        })
                    
                    # Visualization
                    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 10))
                    
                    plot_paths_on_topology(ax1, topology.topology, paths_info, source, destination, disaster_node)
                    ax1.set_title(f'ISP {isp_id} - Normal Paths (ISP Subnet): {source} → {destination}', 
                                 fontsize=16, fontweight='bold', color='darkblue')
                    
                    display_paths_comparison(ax2, paths_info, source, destination, isp_id)
                    
                    plt.tight_layout()
                    plt.show()
                    
                    status_label.value = f"<p style='color: green;'>✅ {len(paths_info)} normal path(s) found in ISP {isp_id} subnet!</p>"
                
            except nx.NetworkXNoPath:
                status_label.value = f"<p style='color: red;'>❌ Nenhum caminho encontrado entre {source} e {destination}!</p>"
            except Exception as e:
                status_label.value = f"<p style='color: red;'>❌ Erro: {str(e)}</p>"
                print(f"❌ Error: {str(e)}")
                import traceback
                traceback.print_exc()
    
    def on_view_mode_change(change):
        """Handle view mode toggle."""
        mode = change['new']
        
        if mode == 'ISP Topology':
            # Show topology view
            output_topology.layout.display = 'block'
            output_paths.layout.display = 'none'
            output_usage.layout.display = 'none'
            output_migration.layout.display = 'none'
            output_criticality.layout.display = 'none'
            output_global_comparison.layout.display = 'none'
            path_controls_row1.layout.display = 'none'
            path_controls_row2.layout.display = 'none'
            topology_controls_row.layout.display = 'flex'
            usage_controls_row.layout.display = 'none'
            migration_controls_row.layout.display = 'none'
            criticality_controls_row.layout.display = 'none'
            global_comparison_controls_row.layout.display = 'none'
            isp_controls_row.layout.display = 'flex'
            update_isp_topology()
        elif mode == 'Path Explorer':
            # Show path explorer view
            output_topology.layout.display = 'none'
            output_paths.layout.display = 'block'
            output_usage.layout.display = 'none'
            output_migration.layout.display = 'none'
            output_criticality.layout.display = 'none'
            output_global_comparison.layout.display = 'none'
            path_controls_row1.layout.display = 'flex'
            path_controls_row2.layout.display = 'flex'
            topology_controls_row.layout.display = 'none'
            usage_controls_row.layout.display = 'none'
            migration_controls_row.layout.display = 'none'
            criticality_controls_row.layout.display = 'none'
            global_comparison_controls_row.layout.display = 'none'
            isp_controls_row.layout.display = 'flex'
            update_path_explorer()
        elif mode == 'ISP Usage Analysis':
            # Show ISP usage analysis view
            output_topology.layout.display = 'none'
            output_paths.layout.display = 'none'
            output_usage.layout.display = 'block'
            output_migration.layout.display = 'none'
            output_criticality.layout.display = 'none'
            output_global_comparison.layout.display = 'none'
            path_controls_row1.layout.display = 'none'
            path_controls_row2.layout.display = 'none'
            topology_controls_row.layout.display = 'none'
            usage_controls_row.layout.display = 'flex'
            migration_controls_row.layout.display = 'none'
            criticality_controls_row.layout.display = 'none'
            global_comparison_controls_row.layout.display = 'none'
            isp_controls_row.layout.display = 'none'
            update_isp_usage_analysis()
        elif mode == 'Migration Analysis':
            # Show migration analysis view
            output_topology.layout.display = 'none'
            output_paths.layout.display = 'none'
            output_usage.layout.display = 'none'
            output_migration.layout.display = 'block'
            output_criticality.layout.display = 'none'
            output_global_comparison.layout.display = 'none'
            path_controls_row1.layout.display = 'none'
            path_controls_row2.layout.display = 'none'
            topology_controls_row.layout.display = 'none'
            usage_controls_row.layout.display = 'none'
            migration_controls_row.layout.display = 'flex'
            criticality_controls_row.layout.display = 'none'
            global_comparison_controls_row.layout.display = 'none'
            isp_controls_row.layout.display = 'none'
            update_migration_analysis()
        elif mode == 'Link Criticality':
            # Show link criticality view
            output_topology.layout.display = 'none'
            output_paths.layout.display = 'none'
            output_usage.layout.display = 'none'
            output_migration.layout.display = 'none'
            output_criticality.layout.display = 'block'
            output_global_comparison.layout.display = 'none'
            path_controls_row1.layout.display = 'none'
            path_controls_row2.layout.display = 'none'
            topology_controls_row.layout.display = 'none'
            usage_controls_row.layout.display = 'none'
            migration_controls_row.layout.display = 'none'
            criticality_controls_row.layout.display = 'flex'
            global_comparison_controls_row.layout.display = 'none'
            isp_controls_row.layout.display = 'none'
            
            update_link_criticality()
        elif mode == 'Global Comparison':
            # Show global comparison view
            output_topology.layout.display = 'none'
            output_paths.layout.display = 'none'
            output_usage.layout.display = 'none'
            output_migration.layout.display = 'none'
            output_criticality.layout.display = 'none'
            output_global_comparison.layout.display = 'block'
            path_controls_row1.layout.display = 'none'
            path_controls_row2.layout.display = 'none'
            topology_controls_row.layout.display = 'none'
            usage_controls_row.layout.display = 'none'
            migration_controls_row.layout.display = 'none'
            criticality_controls_row.layout.display = 'none'
            global_comparison_controls_row.layout.display = 'flex'
            isp_controls_row.layout.display = 'none'
            
            update_global_comparison()
    
    def on_isp_change(change):
        """Handle ISP selection change."""
        # Always update node dropdowns when ISP changes
        update_isp_node_dropdowns()
        
        if view_mode.value == 'ISP Topology':
            update_isp_topology()
        else:
            update_path_explorer()
    
    def on_path_param_change(change):
        """Handle path parameter changes."""
        if view_mode.value == 'Path Explorer':
            update_path_explorer()
    
    def on_link_change(change):
        """Handle link selection change."""
        if view_mode.value == 'ISP Usage Analysis':
            update_isp_usage_analysis()
    
    def on_migration_param_change(change):
        """Handle migration parameter changes."""
        if view_mode.value == 'Migration Analysis':
            update_migration_analysis()
    
    # ========== Attach Callbacks ==========
    view_mode.observe(on_view_mode_change, names='value')
    isp_dropdown.observe(on_isp_change, names='value')
    source_dropdown.observe(on_path_param_change, names='value')
    dest_dropdown.observe(on_path_param_change, names='value')
    k_slider.observe(on_path_param_change, names='value')
    disaster_mode_checkbox.observe(on_path_param_change, names='value')
    show_original_paths_checkbox.observe(on_path_param_change, names='value')
    show_disaster_view_checkbox.observe(lambda change: update_isp_topology(), names='value')
    link_dropdown.observe(on_link_change, names='value')
    migration_paths_slider.observe(on_migration_param_change, names='value')
    
    # Attach observers to ISP checkboxes
    for checkbox in isp_checkboxes.values():
        checkbox.observe(on_migration_param_change, names='value')
    
    # Attach observers to criticality controls
    def on_criticality_param_change(change):
        if view_mode.value == 'Link Criticality':
            update_link_criticality()
    
    def on_criticality_disaster_mode_change(change):
        """Handle disaster mode toggle for criticality view."""
        if view_mode.value == 'Link Criticality':
            # Update visualization
            update_link_criticality()
    
    criticality_disaster_mode_checkbox.observe(on_criticality_disaster_mode_change, names='value')
    
    # Attach observers to global comparison controls
    def on_global_param_change(change):
        """Handle global comparison parameter changes."""
        if view_mode.value == 'Global Comparison':
            update_global_comparison()
    
    global_k_slider.observe(on_global_param_change, names='value')
    for checkbox in global_isp_checkboxes.values():
        checkbox.observe(on_global_param_change, names='value')
    
    recalc_button.on_click(recalculate_weights)
    
    # ========== Layout ==========
    controls_row1 = widgets.HBox([view_mode])
    
    # ISP controls row (only shown for ISP Topology and Path Explorer)
    isp_controls_row = widgets.HBox(
        [isp_dropdown],
        layout=widgets.Layout(display='flex', margin='5px 0')
    )
    
    gui = widgets.VBox([
        controls_row1,
        isp_controls_row,
        weights_info_label,
        weight_params_row,
        topology_controls_row,
        usage_controls_row,
        migration_controls_row,
        criticality_controls_row,
        global_comparison_controls_row,
        path_controls_row1,
        path_controls_row2,
        status_label,
        output_topology,
        output_paths,
        output_usage,
        output_migration,
        output_criticality,
        output_global_comparison
    ])
    
    # ========== Initialize ==========
    update_isp_topology()
    
    return gui


def display_paths_comparison(ax, paths_info, source, destination, isp_id, 
                           original_paths_info=None, show_comparison=False):
    """
    Display paths as an ordered list with comparison statistics.
    
    Args:
        ax: Matplotlib axis
        paths_info: List of path dictionaries with weighted paths
        source: Source node
        destination: Destination node
        isp_id: ISP ID
        original_paths_info: List of unweighted paths for comparison
        show_comparison: Whether to show comparison statistics
    """
    if not paths_info:
        ax.text(0.5, 0.5, 'Nenhum caminho encontrado', 
                ha='center', va='center', fontsize=12, transform=ax.transAxes)
        ax.set_title('Caminhos Ranqueados', fontsize=16, fontweight='bold')
        ax.axis('off')
        return
    
    ax.set_title(f'ISP {isp_id} - Caminhos Ranqueados ({len(paths_info)} encontrados)', 
                 fontsize=16, fontweight='bold')
    ax.axis('off')
    
    # Colors matching the topology plot
    cmap = plt.colormaps['viridis']
    colors = [cmap(i / len(paths_info)) for i in range(len(paths_info))]
    
    y_start = 0.95
    y_step = 0.08
    
    # Show comparison statistics if enabled
    if show_comparison and original_paths_info:
        # Calculate comparison statistics
        weighted_hops = [len(p['caminho']) - 1 for p in paths_info]
        weighted_modulation = [p['fator_de_modulacao'] for p in paths_info]
        weighted_isp_sharing = [p['isp_sharing'] for p in paths_info]
        weighted_distances = [p['weighted_distance'] for p in paths_info]
        
        original_hops = [len(p['caminho']) - 1 for p in original_paths_info]
        original_modulation = [p['fator_de_modulacao'] for p in original_paths_info]
        original_isp_sharing = [p['isp_sharing'] for p in original_paths_info]
        original_distances = [p['weighted_distance'] for p in original_paths_info]
        
        # Calculate averages
        avg_weighted_hops = sum(weighted_hops) / len(weighted_hops)
        avg_weighted_modulation = sum(weighted_modulation) / len(weighted_modulation)
        avg_weighted_isp_sharing = sum(weighted_isp_sharing) / len(weighted_isp_sharing)
        avg_weighted_distance = sum(weighted_distances) / len(weighted_distances)
        
        avg_original_hops = sum(original_hops) / len(original_hops)
        avg_original_modulation = sum(original_modulation) / len(original_modulation)
        avg_original_isp_sharing = sum(original_isp_sharing) / len(original_isp_sharing)
        avg_original_distance = sum(original_distances) / len(original_distances)
        
        # Calculate percentage changes
        hops_change = ((avg_weighted_hops - avg_original_hops) / avg_original_hops * 100) if avg_original_hops > 0 else 0
        modulation_change = ((avg_weighted_modulation - avg_original_modulation) / avg_original_modulation * 100) if avg_original_modulation > 0 else 0
        isp_sharing_change = ((avg_weighted_isp_sharing - avg_original_isp_sharing) / avg_original_isp_sharing * 100) if avg_original_isp_sharing > 0 else 0
        distance_change = ((avg_weighted_distance - avg_original_distance) / avg_original_distance * 100) if avg_original_distance > 0 else 0
        
        # Display comparison box
        ax.text(0.5, y_start, 'Comparison: Weighted vs Unweighted (α=β=γ=0)',
                ha='center', va='top', fontsize=12, fontweight='bold',
                transform=ax.transAxes,
                bbox=dict(boxstyle='round', facecolor='lightblue', alpha=0.8))
        
        y_start -= 0.05
        ax.text(0.1, y_start, f'Average Hops: {avg_original_hops:.1f} → {avg_weighted_hops:.1f} ({hops_change:+.1f}%)',
                fontsize=10, transform=ax.transAxes)
        
        y_start -= 0.04
        ax.text(0.1, y_start, f'Average Modulation: {avg_original_modulation:.1f}x → {avg_weighted_modulation:.1f}x ({modulation_change:+.1f}%)',
                fontsize=10, transform=ax.transAxes)
        
        y_start -= 0.04
        ax.text(0.1, y_start, f'Average ISP Sharing: {avg_original_isp_sharing:.2f} → {avg_weighted_isp_sharing:.2f} ({isp_sharing_change:+.1f}%)',
                fontsize=10, transform=ax.transAxes)
        
        y_start -= 0.04
        ax.text(0.1, y_start, f'Average Weight: {avg_original_distance:.1f} → {avg_weighted_distance:.1f} ({distance_change:+.1f}%)',
                fontsize=10, transform=ax.transAxes)
        
        y_start -= 0.08  # Extra space before paths
    
    # Display paths as ordered list
    for i, (path_info, color) in enumerate(zip(paths_info, colors)):
        path = path_info['caminho']
        hops = len(path) - 1
        distance = path_info['distancia']
        weighted_distance = path_info['weighted_distance']
        modulation = path_info['fator_de_modulacao']
        isp_sharing = path_info['isp_sharing']
        
        # Path number and nodes
        path_str = ' → '.join(map(str, path))
        ax.text(0.05, y_start, f'{i+1}. [{hops}h] {path_str}',
                fontsize=10, fontweight='bold', color=color,
                transform=ax.transAxes)
        
        y_start -= 0.03
        
        # Metrics
        ax.text(0.1, y_start, f'   Distance: {distance:.1f} km | Weighted: {weighted_distance:.1f} | Modulation: {modulation:.1f}x | ISP Sharing: {isp_sharing:.2f}',
                fontsize=9, color=color, style='italic',
                transform=ax.transAxes)
        
        y_start -= 0.05
        
        if y_start < 0.1:  # Prevent text from going off screen
            break


# ============================================================================
# CREATE AND DISPLAY UNIFIED GUI
# ============================================================================
unified_gui = create_unified_isp_gui(
    lista_de_isps,
    topology,
    disaster_node,
    weights_by_link_by_isp
)

display(unified_gui)
